preprocessing, testing for denser pixel grid

In [1]:
import numpy as np
import pandas as pd
import dicom
import os
import matplotlib.pyplot as plt
import cv2
import math

/usr/local/lib/python2.7/dist-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [2]:
IMG_SIZE_PX = 100
SLICE_COUNT = 40

The following chunking methods are based on the Kaggle kernel below, but we optimize the flow and fixed some potential bugs to make it work for general dimensions, such as 100x100x40 or 32x32x32

In [9]:
# By revise the chunk method, we can generate regular number of depth slices consistenly

def chunks(l, s):
    # Credit: Ned Batchelder
    # Link: http://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
    """Yield successive n-sized chunks from l."""
    m = int(math.floor(len(l)/s))
    extra = len(l) - m*s
    b = int(math.floor(extra/2))
    e = extra - b
    loc = 0
    cnt = 0
    print "length={}; hm_slices={};m={}; b={};e={}".format(len(l), s, m, b, e )
    
    while loc < len(l):
        n = m
        if cnt < b or cnt >= s - e:
            n = m + 1
        yield l[loc:loc + n]
        loc += n
        cnt +=1
        #print ("current loc={}, with step= {}, and the current count={}".format(loc, n, cnt))
        
def mean(a):
    return sum(a) / len(a)


def process_data(patient,labels_df,img_px_size=50, hm_slices=20, visualize=False):
    
    label = labels_df.get_value(patient, 'cancer')
    
    path = data_dir + patient
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))

    new_slices = []
    slices = [cv2.resize(np.array(each_slice.pixel_array),(img_px_size,img_px_size), interpolation = cv2.INTER_NEAREST) for each_slice in slices]
    
    #chunk_sizes = int(math.ceil(len(slices) / hm_slices))
    for slice_chunk in chunks(slices, hm_slices):
        slice_chunk = list(map(mean, zip(*slice_chunk)))
        new_slices.append(slice_chunk)

#The manual padding below is not needed after we improve the chunk module, but it does not hurt to keep it here.
    if len(new_slices) == hm_slices-1:
        new_slices.append(new_slices[-1])

    if len(new_slices) == hm_slices-2:
        new_slices.append(new_slices[-1])
        new_slices.append(new_slices[-1])

    if len(new_slices) == hm_slices+2:
        new_val = list(map(mean, zip(*[new_slices[hm_slices-1],new_slices[hm_slices],])))
        del new_slices[hm_slices]
        new_slices[hm_slices-1] = new_val
        
    if len(new_slices) == hm_slices+1:
        new_val = list(map(mean, zip(*[new_slices[hm_slices-1],new_slices[hm_slices],])))
        del new_slices[hm_slices]
        new_slices[hm_slices-1] = new_val

    if visualize:
        fig = plt.figure()
        for num,each_slice in enumerate(new_slices):
            y = fig.add_subplot(4,5,num+1)
            y.imshow(each_slice, cmap='gray')
        plt.show()

    if label == 1: label=np.array([0,1])
    elif label == 0: label=np.array([1,0])
    print "Number of slices origin= {}; number of slices generated = {};".format(len(slices), len(new_slices))    
    
    return np.array(new_slices),label, patient

In [4]:
data_dir = '../data/stage1/'
patients = os.listdir(data_dir)
labels_df = pd.read_csv('../data/stage1_labels.csv', index_col=0)

In [5]:
print labels_df.head()

                                  cancer
id                                      
0015ceb851d7251b8f399e39779d1e7d       1
0030a160d58723ff36d73f41b170ec21       0
003f41c78e6acfa92430a057ac0b306e       0
006b96310a37b36cccb2ab48d10b49a3       1
008464bb8521d09a42985dd8add3d0d2       1


In [10]:
#Test the new flow, checking if the number of the slices are constantly equal to SLICE_COUNT
much_data2 = []

cnt = 1
for num,patient in enumerate(patients):
    if num % 100 == 0:
        print(num)
    try:
        img_data,label, pID = process_data(patient,labels_df,img_px_size=IMG_SIZE_PX, hm_slices=SLICE_COUNT)
        if img_data.shape[0] != SLICE_COUNT:
            print(img_data.shape,label)
        much_data2.append([img_data,label, pID])
        cnt += 1
    except KeyError as e:
        print('This is unlabeled data {}!'.format(num))


0
This is unlabeled data 0!


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:30: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


length=234; hm_slices=40;m=5; b=17;e=17
Number of slices origin= 234; number of slices generated = 40;
This is unlabeled data 2!
length=113; hm_slices=40;m=2; b=16;e=17
Number of slices origin= 113; number of slices generated = 40;
length=177; hm_slices=40;m=4; b=8;e=9
Number of slices origin= 177; number of slices generated = 40;
length=154; hm_slices=40;m=3; b=17;e=17
Number of slices origin= 154; number of slices generated = 40;
length=158; hm_slices=40;m=3; b=19;e=19
Number of slices origin= 158; number of slices generated = 40;
length=144; hm_slices=40;m=3; b=12;e=12
Number of slices origin= 144; number of slices generated = 40;
length=115; hm_slices=40;m=2; b=17;e=18
Number of slices origin= 115; number of slices generated = 40;
length=162; hm_slices=40;m=4; b=1;e=1
Number of slices origin= 162; number of slices generated = 40;
This is unlabeled data 10!
length=175; hm_slices=40;m=4; b=7;e=8
Number of slices origin= 175; number of slices generated = 40;
length=147; hm_slices=40;m

In [12]:
len(much_data2[1][0][23])

100

In [14]:
# Do not be scared if you see a lot of -2000 or -1000, they are corresponding to the edge of the images
# let us look into a slice in the middle:
much_data2[1][0][23][51]

array([ 909,  921,  929,  934,  925,  917,  927,  920,  935, 1017, 1052,
       1062, 1102, 1098,  177,  168,  185,  204,  336,  273,  157,  155,
        157,  188,  315,  213,  232,  187,  212,  295,  246,  217,  172,
        118,  109,  111,  115,  101,  224,  426,  576,  438,  547,  562,
        265,  265,  169,  256,  901, 1061, 1050, 1064, 1056, 1073, 1063,
       1048, 1067, 1059, 1065, 1087, 1092, 1064, 1061, 1056, 1052,  997,
       1022, 1043,  985,  998,  850,  349,  181,  167,  211,  241,  279,
        223,  312,  205,  162,  179,  209,  168,  173,  216,  177,  185,
       1057, 1181, 1072, 1023,  932,  937,  932,  920,  935,  920,  914,
        916], dtype=int16)

In [15]:
#save the numpy data for modeling input
np.save('../data/muchdata2-{}-{}-{}.npy'.format(IMG_SIZE_PX,IMG_SIZE_PX,SLICE_COUNT), much_data2)

length=234; hm_slices=20;m=11; b=7;e=7
length origin= 234; length generated = 20;
This is unlabeled data 2!
length=113; hm_slices=20;m=5; b=6;e=7
length origin= 113; length generated = 20;
length=177; hm_slices=20;m=8; b=8;e=9
length origin= 177; length generated = 20;
length=154; hm_slices=20;m=7; b=7;e=7
length origin= 154; length generated = 20;
length=158; hm_slices=20;m=7; b=9;e=9
length origin= 158; length generated = 20;
length=144; hm_slices=20;m=7; b=2;e=2
length origin= 144; length generated = 20;
length=115; hm_slices=20;m=5; b=7;e=8
length origin= 115; length generated = 20;
length=162; hm_slices=20;m=8; b=1;e=1
length origin= 162; length generated = 20;
This is unlabeled data 10!
length=175; hm_slices=20;m=8; b=7;e=8
length origin= 175; length generated = 20;
length=147; hm_slices=20;m=7; b=3;e=4
length origin= 147; length generated = 20;
length=132; hm_slices=20;m=6; b=6;e=6
length origin= 132; length generated = 20;
length=163; hm_slices=20;m=8; b=1;e=2
length origin= 163; length generated = 20;
This is unlabeled data 15!
length=128; hm_slices=20;m=6; b=4;e=4
length origin= 128; length generated = 20;
length=364; hm_slices=20;m=18; b=2;e=2
length origin= 364; length generated = 20;
length=126; hm_slices=20;m=6; b=3;e=3
length origin= 126; length generated = 20;
length=231; hm_slices=20;m=11; b=5;e=6
length origin= 231; length generated = 20;
length=162; hm_slices=20;m=8; b=1;e=1
length origin= 162; length generated = 20;
This is unlabeled data 21!
length=146; hm_slices=20;m=7; b=3;e=3
length origin= 146; length generated = 20;
length=134; hm_slices=20;m=6; b=7;e=7
length origin= 134; length generated = 20;
This is unlabeled data 24!
length=167; hm_slices=20;m=8; b=3;e=4
length origin= 167; length generated = 20;
length=148; hm_slices=20;m=7; b=4;e=4
length origin= 148; length generated = 20;
length=125; hm_slices=20;m=6; b=2;e=3
length origin= 125; length generated = 20;
length=146; hm_slices=20;m=7; b=3;e=3
length origin= 146; length generated = 20;
length=179; hm_slices=20;m=8; b=9;e=10
length origin= 179; length generated = 20;
length=188; hm_slices=20;m=9; b=4;e=4
length origin= 188; length generated = 20;
length=279; hm_slices=20;m=13; b=9;e=10
length origin= 279; length generated = 20;
length=167; hm_slices=20;m=8; b=3;e=4
length origin= 167; length generated = 20;
length=163; hm_slices=20;m=8; b=1;e=2
length origin= 163; length generated = 20;
This is unlabeled data 34!
length=337; hm_slices=20;m=16; b=8;e=9
length origin= 337; length generated = 20;
length=251; hm_slices=20;m=12; b=5;e=6
length origin= 251; length generated = 20;
length=220; hm_slices=20;m=11; b=0;e=0
length origin= 220; length generated = 20;
length=142; hm_slices=20;m=7; b=1;e=1
length origin= 142; length generated = 20;
length=124; hm_slices=20;m=6; b=2;e=2
length origin= 124; length generated = 20;
length=160; hm_slices=20;m=8; b=0;e=0
length origin= 160; length generated = 20;
length=225; hm_slices=20;m=11; b=2;e=3
length origin= 225; length generated = 20;
length=184; hm_slices=20;m=9; b=2;e=2
length origin= 184; length generated = 20;
length=183; hm_slices=20;m=9; b=1;e=2
length origin= 183; length generated = 20;
length=134; hm_slices=20;m=6; b=7;e=7
length origin= 134; length generated = 20;
length=173; hm_slices=20;m=8; b=6;e=7
length origin= 173; length generated = 20;
length=191; hm_slices=20;m=9; b=5;e=6
length origin= 191; length generated = 20;
length=161; hm_slices=20;m=8; b=0;e=1
length origin= 161; length generated = 20;
length=156; hm_slices=20;m=7; b=8;e=8
length origin= 156; length generated = 20;
length=171; hm_slices=20;m=8; b=5;e=6
length origin= 171; length generated = 20;
length=147; hm_slices=20;m=7; b=3;e=4
length origin= 147; length generated = 20;
length=128; hm_slices=20;m=6; b=4;e=4
length origin= 128; length generated = 20;
length=147; hm_slices=20;m=7; b=3;e=4
length origin= 147; length generated = 20;
length=110; hm_slices=20;m=5; b=5;e=5
length origin= 110; length generated = 20;
This is unlabeled data 54!
length=156; hm_slices=20;m=7; b=8;e=8
length origin= 156; length generated = 20;
length=157; hm_slices=20;m=7; b=8;e=9
length origin= 157; length generated = 20;
length=128; hm_slices=20;m=6; b=4;e=4
length origin= 128; length generated = 20;
length=163; hm_slices=20;m=8; b=1;e=2
length origin= 163; length generated = 20;
length=143; hm_slices=20;m=7; b=1;e=2
length origin= 143; length generated = 20;
length=112; hm_slices=20;m=5; b=6;e=6
length origin= 112; length generated = 20;
length=123; hm_slices=20;m=6; b=1;e=2
length origin= 123; length generated = 20;
length=177; hm_slices=20;m=8; b=8;e=9
length origin= 177; length generated = 20;
length=318; hm_slices=20;m=15; b=9;e=9
length origin= 318; length generated = 20;
length=136; hm_slices=20;m=6; b=8;e=8
length origin= 136; length generated = 20;
length=224; hm_slices=20;m=11; b=2;e=2
length origin= 224; length generated = 20;
This is unlabeled data 66!
length=133; hm_slices=20;m=6; b=6;e=7
length origin= 133; length generated = 20;
length=178; hm_slices=20;m=8; b=9;e=9
length origin= 178; length generated = 20;
length=131; hm_slices=20;m=6; b=5;e=6
length origin= 131; length generated = 20;
length=131; hm_slices=20;m=6; b=5;e=6
length origin= 131; length generated = 20;
length=174; hm_slices=20;m=8; b=7;e=7
length origin= 174; length generated = 20;
length=196; hm_slices=20;m=9; b=8;e=8
length origin= 196; length generated = 20;
length=298; hm_slices=20;m=14; b=9;e=9
length origin= 298; length generated = 20;
length=132; hm_slices=20;m=6; b=6;e=6
length origin= 132; length generated = 20;
length=242; hm_slices=20;m=12; b=1;e=1
length origin= 242; length generated = 20;
length=352; hm_slices=20;m=17; b=6;e=6
length origin= 352; length generated = 20;
length=203; hm_slices=20;m=10; b=1;e=2
length origin= 203; length generated = 20;
length=144; hm_slices=20;m=7; b=2;e=2
length origin= 144; length generated = 20;
length=166; hm_slices=20;m=8; b=3;e=3
length origin= 166; length generated = 20;
length=166; hm_slices=20;m=8; b=3;e=3
length origin= 166; length generated = 20;
length=187; hm_slices=20;m=9; b=3;e=4
length origin= 187; length generated = 20;
length=153; hm_slices=20;m=7; b=6;e=7
length origin= 153; length generated = 20;
length=173; hm_slices=20;m=8; b=6;e=7
length origin= 173; length generated = 20;
length=125; hm_slices=20;m=6; b=2;e=3
length origin= 125; length generated = 20;
length=121; hm_slices=20;m=6; b=0;e=1
length origin= 121; length generated = 20;
length=205; hm_slices=20;m=10; b=2;e=3
length origin= 205; length generated = 20;
length=146; hm_slices=20;m=7; b=3;e=3
length origin= 146; length generated = 20;
This is unlabeled data 88!
length=179; hm_slices=20;m=8; b=9;e=10
length origin= 179; length generated = 20;
This is unlabeled data 90!
length=154; hm_slices=20;m=7; b=7;e=7
length origin= 154; length generated = 20;
length=183; hm_slices=20;m=9; b=1;e=2
length origin= 183; length generated = 20;
length=280; hm_slices=20;m=14; b=0;e=0
length origin= 280; length generated = 20;
length=278; hm_slices=20;m=13; b=9;e=9
length origin= 278; length generated = 20;
length=218; hm_slices=20;m=10; b=9;e=9
length origin= 218; length generated = 20;
length=177; hm_slices=20;m=8; b=8;e=9
length origin= 177; length generated = 20;
length=122; hm_slices=20;m=6; b=1;e=1
length origin= 122; length generated = 20;
length=259; hm_slices=20;m=12; b=9;e=10
length origin= 259; length generated = 20;
length=168; hm_slices=20;m=8; b=4;e=4
length origin= 168; length generated = 20;
100
length=282; hm_slices=20;m=14; b=1;e=1
length origin= 282; length generated = 20;
length=117; hm_slices=20;m=5; b=8;e=9
length origin= 117; length generated = 20;
length=212; hm_slices=20;m=10; b=6;e=6
length origin= 212; length generated = 20;
length=498; hm_slices=20;m=24; b=9;e=9
length origin= 498; length generated = 20;
This is unlabeled data 104!
length=247; hm_slices=20;m=12; b=3;e=4
length origin= 247; length generated = 20;
length=200; hm_slices=20;m=10; b=0;e=0
length origin= 200; length generated = 20;
length=169; hm_slices=20;m=8; b=4;e=5
length origin= 169; length generated = 20;
length=163; hm_slices=20;m=8; b=1;e=2
length origin= 163; length generated = 20;
length=122; hm_slices=20;m=6; b=1;e=1
length origin= 122; length generated = 20;

length=250; hm_slices=20;m=12; b=5;e=5
length origin= 250; length generated = 20;
length=143; hm_slices=20;m=7; b=1;e=2
length origin= 143; length generated = 20;
length=174; hm_slices=20;m=8; b=7;e=7
length origin= 174; length generated = 20;
length=188; hm_slices=20;m=9; b=4;e=4
length origin= 188; length generated = 20;
length=198; hm_slices=20;m=9; b=9;e=9
length origin= 198; length generated = 20;
length=388; hm_slices=20;m=19; b=4;e=4
length origin= 388; length generated = 20;
length=154; hm_slices=20;m=7; b=7;e=7
length origin= 154; length generated = 20;
length=209; hm_slices=20;m=10; b=4;e=5
length origin= 209; length generated = 20;
length=142; hm_slices=20;m=7; b=1;e=1
length origin= 142; length generated = 20;
length=177; hm_slices=20;m=8; b=8;e=9
length origin= 177; length generated = 20;
length=147; hm_slices=20;m=7; b=3;e=4
length origin= 147; length generated = 20;
length=150; hm_slices=20;m=7; b=5;e=5
length origin= 150; length generated = 20;
This is unlabeled data 122!
length=138; hm_slices=20;m=6; b=9;e=9
length origin= 138; length generated = 20;
length=162; hm_slices=20;m=8; b=1;e=1
length origin= 162; length generated = 20;
length=146; hm_slices=20;m=7; b=3;e=3
length origin= 146; length generated = 20;
length=153; hm_slices=20;m=7; b=6;e=7
length origin= 153; length generated = 20;
length=103; hm_slices=20;m=5; b=1;e=2
length origin= 103; length generated = 20;
This is unlabeled data 128!
length=487; hm_slices=20;m=24; b=3;e=4
length origin= 487; length generated = 20;
length=180; hm_slices=20;m=9; b=0;e=0
length origin= 180; length generated = 20;
length=149; hm_slices=20;m=7; b=4;e=5
length origin= 149; length generated = 20;
length=231; hm_slices=20;m=11; b=5;e=6
length origin= 231; length generated = 20;
length=172; hm_slices=20;m=8; b=6;e=6
length origin= 172; length generated = 20;
length=117; hm_slices=20;m=5; b=8;e=9
length origin= 117; length generated = 20;
length=187; hm_slices=20;m=9; b=3;e=4
length origin= 187; length generated = 20;
length=172; hm_slices=20;m=8; b=6;e=6
length origin= 172; length generated = 20;
length=275; hm_slices=20;m=13; b=7;e=8
length origin= 275; length generated = 20;
This is unlabeled data 138!
length=323; hm_slices=20;m=16; b=1;e=2
length origin= 323; length generated = 20;
length=149; hm_slices=20;m=7; b=4;e=5
length origin= 149; length generated = 20;
length=132; hm_slices=20;m=6; b=6;e=6
length origin= 132; length generated = 20;
length=256; hm_slices=20;m=12; b=8;e=8
length origin= 256; length generated = 20;
length=146; hm_slices=20;m=7; b=3;e=3
length origin= 146; length generated = 20;
length=124; hm_slices=20;m=6; b=2;e=2
length origin= 124; length generated = 20;
length=244; hm_slices=20;m=12; b=2;e=2
length origin= 244; length generated = 20;
length=114; hm_slices=20;m=5; b=7;e=7
length origin= 114; length generated = 20;
length=306; hm_slices=20;m=15; b=3;e=3
length origin= 306; length generated = 20;
length=139; hm_slices=20;m=6; b=9;e=10
length origin= 139; length generated = 20;
length=348; hm_slices=20;m=17; b=4;e=4
length origin= 348; length generated = 20;
This is unlabeled data 150!
This is unlabeled data 151!
length=149; hm_slices=20;m=7; b=4;e=5
length origin= 149; length generated = 20;
length=145; hm_slices=20;m=7; b=2;e=3
length origin= 145; length generated = 20;
length=138; hm_slices=20;m=6; b=9;e=9
length origin= 138; length generated = 20;
length=162; hm_slices=20;m=8; b=1;e=1
length origin= 162; length generated = 20;
length=151; hm_slices=20;m=7; b=5;e=6
length origin= 151; length generated = 20;
length=262; hm_slices=20;m=13; b=1;e=1
length origin= 262; length generated = 20;
This is unlabeled data 158!
length=185; hm_slices=20;m=9; b=2;e=3
length origin= 185; length generated = 20;
length=203; hm_slices=20;m=10; b=1;e=2
length origin= 203; length generated = 20;
This is unlabeled data 161!
length=193; hm_slices=20;m=9; b=6;e=7
length origin= 193; length generated = 20;
length=144; hm_slices=20;m=7; b=2;e=2
length origin= 144; length generated = 20;
length=345; hm_slices=20;m=17; b=2;e=3
length origin= 345; length generated = 20;
length=146; hm_slices=20;m=7; b=3;e=3
length origin= 146; length generated = 20;
length=164; hm_slices=20;m=8; b=2;e=2
length origin= 164; length generated = 20;
length=241; hm_slices=20;m=12; b=0;e=1
length origin= 241; length generated = 20;
This is unlabeled data 168!
length=176; hm_slices=20;m=8; b=8;e=8
length origin= 176; length generated = 20;
length=219; hm_slices=20;m=10; b=9;e=10
length origin= 219; length generated = 20;
length=193; hm_slices=20;m=9; b=6;e=7
length origin= 193; length generated = 20;
length=205; hm_slices=20;m=10; b=2;e=3
length origin= 205; length generated = 20;
length=298; hm_slices=20;m=14; b=9;e=9
length origin= 298; length generated = 20;
This is unlabeled data 174!
length=259; hm_slices=20;m=12; b=9;e=10
length origin= 259; length generated = 20;
length=123; hm_slices=20;m=6; b=1;e=2
length origin= 123; length generated = 20;
length=131; hm_slices=20;m=6; b=5;e=6
length origin= 131; length generated = 20;
length=136; hm_slices=20;m=6; b=8;e=8
length origin= 136; length generated = 20;
length=115; hm_slices=20;m=5; b=7;e=8
length origin= 115; length generated = 20;
length=131; hm_slices=20;m=6; b=5;e=6
length origin= 131; length generated = 20;
length=156; hm_slices=20;m=7; b=8;e=8
length origin= 156; length generated = 20;
length=282; hm_slices=20;m=14; b=1;e=1
length origin= 282; length generated = 20;
length=156; hm_slices=20;m=7; b=8;e=8
length origin= 156; length generated = 20;
length=178; hm_slices=20;m=8; b=9;e=9
length origin= 178; length generated = 20;
length=184; hm_slices=20;m=9; b=2;e=2
length origin= 184; length generated = 20;
This is unlabeled data 186!
length=503; hm_slices=20;m=25; b=1;e=2
length origin= 503; length generated = 20;
length=162; hm_slices=20;m=8; b=1;e=1
length origin= 162; length generated = 20;
length=165; hm_slices=20;m=8; b=2;e=3
length origin= 165; length generated = 20;
length=271; hm_slices=20;m=13; b=5;e=6
length origin= 271; length generated = 20;
length=116; hm_slices=20;m=5; b=8;e=8
length origin= 116; length generated = 20;
length=189; hm_slices=20;m=9; b=4;e=5
length origin= 189; length generated = 20;
length=140; hm_slices=20;m=7; b=0;e=0
length origin= 140; length generated = 20;
length=161; hm_slices=20;m=8; b=0;e=1
length origin= 161; length generated = 20;
length=134; hm_slices=20;m=6; b=7;e=7
length origin= 134; length generated = 20;
This is unlabeled data 196!
length=157; hm_slices=20;m=7; b=8;e=9
length origin= 157; length generated = 20;
This is unlabeled data 198!
length=169; hm_slices=20;m=8; b=4;e=5
length origin= 169; length generated = 20;
200
length=184; hm_slices=20;m=9; b=2;e=2
length origin= 184; length generated = 20;
length=277; hm_slices=20;m=13; b=8;e=9
length origin= 277; length generated = 20;
length=159; hm_slices=20;m=7; b=9;e=10
length origin= 159; length generated = 20;
length=151; hm_slices=20;m=7; b=5;e=6
length origin= 151; length generated = 20;
length=118; hm_slices=20;m=5; b=9;e=9
length origin= 118; length generated = 20;
length=176; hm_slices=20;m=8; b=8;e=8
length origin= 176; length generated = 20;
length=144; hm_slices=20;m=7; b=2;e=2
length origin= 144; length generated = 20;
length=266; hm_slices=20;m=13; b=3;e=3
length origin= 266; length generated = 20;
length=175; hm_slices=20;m=8; b=7;e=8
length origin= 175; length generated = 20;
This is unlabeled data 209!
length=185; hm_slices=20;m=9; b=2;e=3
length origin= 185; length generated = 20;
length=111; hm_slices=20;m=5; b=5;e=6
length origin= 111; length generated = 20;
length=160; hm_slices=20;m=8; b=0;e=0
length origin= 160; length generated = 20;
length=123; hm_slices=20;m=6; b=1;e=2
length origin= 123; length generated = 20;
length=128; hm_slices=20;m=6; b=4;e=4
length origin= 128; length generated = 20;
length=147; hm_slices=20;m=7; b=3;e=4
length origin= 147; length generated = 20;
length=126; hm_slices=20;m=6; b=3;e=3
length origin= 126; length generated = 20;
This is unlabeled data 217!
length=280; hm_slices=20;m=14; b=0;e=0
length origin= 280; length generated = 20;
length=518; hm_slices=20;m=25; b=9;e=9
length origin= 518; length generated = 20;

length=169; hm_slices=20;m=8; b=4;e=5
length origin= 169; length generated = 20;
length=262; hm_slices=20;m=13; b=1;e=1
length origin= 262; length generated = 20;
length=113; hm_slices=20;m=5; b=6;e=7
length origin= 113; length generated = 20;
length=354; hm_slices=20;m=17; b=7;e=7
length origin= 354; length generated = 20;
length=115; hm_slices=20;m=5; b=7;e=8
length origin= 115; length generated = 20;
This is unlabeled data 225!
length=131; hm_slices=20;m=6; b=5;e=6
length origin= 131; length generated = 20;
length=162; hm_slices=20;m=8; b=1;e=1
length origin= 162; length generated = 20;
length=132; hm_slices=20;m=6; b=6;e=6
length origin= 132; length generated = 20;
length=146; hm_slices=20;m=7; b=3;e=3
length origin= 146; length generated = 20;
length=151; hm_slices=20;m=7; b=5;e=6
length origin= 151; length generated = 20;
length=174; hm_slices=20;m=8; b=7;e=7
length origin= 174; length generated = 20;
length=163; hm_slices=20;m=8; b=1;e=2
length origin= 163; length generated = 20;
length=134; hm_slices=20;m=6; b=7;e=7
length origin= 134; length generated = 20;
This is unlabeled data 234!
length=211; hm_slices=20;m=10; b=5;e=6
length origin= 211; length generated = 20;
length=101; hm_slices=20;m=5; b=0;e=1
length origin= 101; length generated = 20;
length=116; hm_slices=20;m=5; b=8;e=8
length origin= 116; length generated = 20;
length=159; hm_slices=20;m=7; b=9;e=10
length origin= 159; length generated = 20;
length=175; hm_slices=20;m=8; b=7;e=8
length origin= 175; length generated = 20;
length=127; hm_slices=20;m=6; b=3;e=4
length origin= 127; length generated = 20;
length=167; hm_slices=20;m=8; b=3;e=4
length origin= 167; length generated = 20;
length=167; hm_slices=20;m=8; b=3;e=4
length origin= 167; length generated = 20;
length=156; hm_slices=20;m=7; b=8;e=8
length origin= 156; length generated = 20;
This is unlabeled data 244!
length=181; hm_slices=20;m=9; b=0;e=1
length origin= 181; length generated = 20;
length=194; hm_slices=20;m=9; b=7;e=7
length origin= 194; length generated = 20;
length=196; hm_slices=20;m=9; b=8;e=8
length origin= 196; length generated = 20;
length=181; hm_slices=20;m=9; b=0;e=1
length origin= 181; length generated = 20;
length=126; hm_slices=20;m=6; b=3;e=3
length origin= 126; length generated = 20;
length=167; hm_slices=20;m=8; b=3;e=4
length origin= 167; length generated = 20;
length=208; hm_slices=20;m=10; b=4;e=4
length origin= 208; length generated = 20;
length=160; hm_slices=20;m=8; b=0;e=0
length origin= 160; length generated = 20;
length=181; hm_slices=20;m=9; b=0;e=1
length origin= 181; length generated = 20;
length=169; hm_slices=20;m=8; b=4;e=5
length origin= 169; length generated = 20;
length=541; hm_slices=20;m=27; b=0;e=1
length origin= 541; length generated = 20;
length=190; hm_slices=20;m=9; b=5;e=5
length origin= 190; length generated = 20;
length=304; hm_slices=20;m=15; b=2;e=2
length origin= 304; length generated = 20;
This is unlabeled data 258!
length=371; hm_slices=20;m=18; b=5;e=6
length origin= 371; length generated = 20;
length=167; hm_slices=20;m=8; b=3;e=4
length origin= 167; length generated = 20;
length=155; hm_slices=20;m=7; b=7;e=8
length origin= 155; length generated = 20;
length=130; hm_slices=20;m=6; b=5;e=5
length origin= 130; length generated = 20;
length=107; hm_slices=20;m=5; b=3;e=4
length origin= 107; length generated = 20;
length=141; hm_slices=20;m=7; b=0;e=1
length origin= 141; length generated = 20;
length=124; hm_slices=20;m=6; b=2;e=2
length origin= 124; length generated = 20;
length=134; hm_slices=20;m=6; b=7;e=7
length origin= 134; length generated = 20;
length=154; hm_slices=20;m=7; b=7;e=7
length origin= 154; length generated = 20;
length=150; hm_slices=20;m=7; b=5;e=5
length origin= 150; length generated = 20;
length=152; hm_slices=20;m=7; b=6;e=6
length origin= 152; length generated = 20;
This is unlabeled data 270!
This is unlabeled data 271!
length=123; hm_slices=20;m=6; b=1;e=2
length origin= 123; length generated = 20;
length=162; hm_slices=20;m=8; b=1;e=1
length origin= 162; length generated = 20;
length=171; hm_slices=20;m=8; b=5;e=6
length origin= 171; length generated = 20;
length=206; hm_slices=20;m=10; b=3;e=3
length origin= 206; length generated = 20;
length=162; hm_slices=20;m=8; b=1;e=1
length origin= 162; length generated = 20;
length=119; hm_slices=20;m=5; b=9;e=10
length origin= 119; length generated = 20;
length=149; hm_slices=20;m=7; b=4;e=5
length origin= 149; length generated = 20;
length=140; hm_slices=20;m=7; b=0;e=0
length origin= 140; length generated = 20;
length=153; hm_slices=20;m=7; b=6;e=7
length origin= 153; length generated = 20;
length=115; hm_slices=20;m=5; b=7;e=8
length origin= 115; length generated = 20;
length=297; hm_slices=20;m=14; b=8;e=9
length origin= 297; length generated = 20;
length=231; hm_slices=20;m=11; b=5;e=6
length origin= 231; length generated = 20;
length=186; hm_slices=20;m=9; b=3;e=3
length origin= 186; length generated = 20;
length=252; hm_slices=20;m=12; b=6;e=6
length origin= 252; length generated = 20;
length=117; hm_slices=20;m=5; b=8;e=9
length origin= 117; length generated = 20;
length=121; hm_slices=20;m=6; b=0;e=1
length origin= 121; length generated = 20;
length=233; hm_slices=20;m=11; b=6;e=7
length origin= 233; length generated = 20;
length=162; hm_slices=20;m=8; b=1;e=1
length origin= 162; length generated = 20;
This is unlabeled data 290!
This is unlabeled data 291!
This is unlabeled data 292!
length=152; hm_slices=20;m=7; b=6;e=6
length origin= 152; length generated = 20;
This is unlabeled data 294!
length=111; hm_slices=20;m=5; b=5;e=6
length origin= 111; length generated = 20;
length=125; hm_slices=20;m=6; b=2;e=3
length origin= 125; length generated = 20;
length=151; hm_slices=20;m=7; b=5;e=6
length origin= 151; length generated = 20;
length=126; hm_slices=20;m=6; b=3;e=3
length origin= 126; length generated = 20;
This is unlabeled data 299!
300
length=267; hm_slices=20;m=13; b=3;e=4
length origin= 267; length generated = 20;
length=173; hm_slices=20;m=8; b=6;e=7
length origin= 173; length generated = 20;
length=134; hm_slices=20;m=6; b=7;e=7
length origin= 134; length generated = 20;
length=192; hm_slices=20;m=9; b=6;e=6
length origin= 192; length generated = 20;
length=214; hm_slices=20;m=10; b=7;e=7
length origin= 214; length generated = 20;
length=124; hm_slices=20;m=6; b=2;e=2
length origin= 124; length generated = 20;
length=158; hm_slices=20;m=7; b=9;e=9
length origin= 158; length generated = 20;
length=151; hm_slices=20;m=7; b=5;e=6
length origin= 151; length generated = 20;
This is unlabeled data 308!
length=159; hm_slices=20;m=7; b=9;e=10
length origin= 159; length generated = 20;
length=96; hm_slices=20;m=4; b=8;e=8
length origin= 96; length generated = 20;
length=161; hm_slices=20;m=8; b=0;e=1
length origin= 161; length generated = 20;
length=162; hm_slices=20;m=8; b=1;e=1
length origin= 162; length generated = 20;
length=154; hm_slices=20;m=7; b=7;e=7
length origin= 154; length generated = 20;
length=129; hm_slices=20;m=6; b=4;e=5
length origin= 129; length generated = 20;
length=134; hm_slices=20;m=6; b=7;e=7
length origin= 134; length generated = 20;
length=148; hm_slices=20;m=7; b=4;e=4
length origin= 148; length generated = 20;
length=185; hm_slices=20;m=9; b=2;e=3
length origin= 185; length generated = 20;
This is unlabeled data 318!
length=163; hm_slices=20;m=8; b=1;e=2
length origin= 163; length generated = 20;
length=203; hm_slices=20;m=10; b=1;e=2
length origin= 203; length generated = 20;
This is unlabeled data 321!
length=116; hm_slices=20;m=5; b=8;e=8
length origin= 116; length generated = 20;
length=141; hm_slices=20;m=7; b=0;e=1
length origin= 141; length generated = 20;
This is unlabeled data 324!
length=175; hm_slices=20;m=8; b=7;e=8
length origin= 175; length generated = 20;
length=171; hm_slices=20;m=8; b=5;e=6
length origin= 171; length generated = 20;
length=177; hm_slices=20;m=8; b=8;e=9
length origin= 177; length generated = 20;
length=124; hm_slices=20;m=6; b=2;e=2
length origin= 124; length generated = 20;
length=130; hm_slices=20;m=6; b=5;e=5
length origin= 130; length generated = 20;
length=133; hm_slices=20;m=6; b=6;e=7
length origin= 133; length generated = 20;

length=241; hm_slices=20;m=12; b=0;e=1
length origin= 241; length generated = 20;
length=122; hm_slices=20;m=6; b=1;e=1
length origin= 122; length generated = 20;
This is unlabeled data 333!
length=156; hm_slices=20;m=7; b=8;e=8
length origin= 156; length generated = 20;
length=144; hm_slices=20;m=7; b=2;e=2
length origin= 144; length generated = 20;
This is unlabeled data 336!
length=154; hm_slices=20;m=7; b=7;e=7
length origin= 154; length generated = 20;
length=183; hm_slices=20;m=9; b=1;e=2
length origin= 183; length generated = 20;
length=129; hm_slices=20;m=6; b=4;e=5
length origin= 129; length generated = 20;
length=328; hm_slices=20;m=16; b=4;e=4
length origin= 328; length generated = 20;
length=245; hm_slices=20;m=12; b=2;e=3
length origin= 245; length generated = 20;
length=191; hm_slices=20;m=9; b=5;e=6
length origin= 191; length generated = 20;
This is unlabeled data 343!
length=189; hm_slices=20;m=9; b=4;e=5
length origin= 189; length generated = 20;
length=175; hm_slices=20;m=8; b=7;e=8
length origin= 175; length generated = 20;
This is unlabeled data 346!
length=162; hm_slices=20;m=8; b=1;e=1
length origin= 162; length generated = 20;
length=173; hm_slices=20;m=8; b=6;e=7
length origin= 173; length generated = 20;
length=155; hm_slices=20;m=7; b=7;e=8
length origin= 155; length generated = 20;
length=115; hm_slices=20;m=5; b=7;e=8
length origin= 115; length generated = 20;
length=162; hm_slices=20;m=8; b=1;e=1
length origin= 162; length generated = 20;
length=157; hm_slices=20;m=7; b=8;e=9
length origin= 157; length generated = 20;
length=122; hm_slices=20;m=6; b=1;e=1
length origin= 122; length generated = 20;
length=130; hm_slices=20;m=6; b=5;e=5
length origin= 130; length generated = 20;
length=145; hm_slices=20;m=7; b=2;e=3
length origin= 145; length generated = 20;
length=170; hm_slices=20;m=8; b=5;e=5
length origin= 170; length generated = 20;
length=133; hm_slices=20;m=6; b=6;e=7
length origin= 133; length generated = 20;
length=142; hm_slices=20;m=7; b=1;e=1
length origin= 142; length generated = 20;
length=156; hm_slices=20;m=7; b=8;e=8
length origin= 156; length generated = 20;
length=520; hm_slices=20;m=26; b=0;e=0
length origin= 520; length generated = 20;
length=133; hm_slices=20;m=6; b=6;e=7
length origin= 133; length generated = 20;
length=172; hm_slices=20;m=8; b=6;e=6
length origin= 172; length generated = 20;
length=340; hm_slices=20;m=17; b=0;e=0
length origin= 340; length generated = 20;
length=95; hm_slices=20;m=4; b=7;e=8
length origin= 95; length generated = 20;
length=165; hm_slices=20;m=8; b=2;e=3
length origin= 165; length generated = 20;
length=153; hm_slices=20;m=7; b=6;e=7
length origin= 153; length generated = 20;
length=397; hm_slices=20;m=19; b=8;e=9
length origin= 397; length generated = 20;
length=165; hm_slices=20;m=8; b=2;e=3
length origin= 165; length generated = 20;
This is unlabeled data 369!
length=163; hm_slices=20;m=8; b=1;e=2
length origin= 163; length generated = 20;
length=126; hm_slices=20;m=6; b=3;e=3
length origin= 126; length generated = 20;
length=155; hm_slices=20;m=7; b=7;e=8
length origin= 155; length generated = 20;
length=216; hm_slices=20;m=10; b=8;e=8
length origin= 216; length generated = 20;
length=170; hm_slices=20;m=8; b=5;e=5
length origin= 170; length generated = 20;
This is unlabeled data 375!
length=159; hm_slices=20;m=7; b=9;e=10
length origin= 159; length generated = 20;
length=123; hm_slices=20;m=6; b=1;e=2
length origin= 123; length generated = 20;
length=188; hm_slices=20;m=9; b=4;e=4
length origin= 188; length generated = 20;
length=220; hm_slices=20;m=11; b=0;e=0
length origin= 220; length generated = 20;
length=106; hm_slices=20;m=5; b=3;e=3
length origin= 106; length generated = 20;
length=153; hm_slices=20;m=7; b=6;e=7
length origin= 153; length generated = 20;
length=175; hm_slices=20;m=8; b=7;e=8
length origin= 175; length generated = 20;
length=147; hm_slices=20;m=7; b=3;e=4
length origin= 147; length generated = 20;
This is unlabeled data 384!
length=164; hm_slices=20;m=8; b=2;e=2
length origin= 164; length generated = 20;
length=164; hm_slices=20;m=8; b=2;e=2
length origin= 164; length generated = 20;
length=157; hm_slices=20;m=7; b=8;e=9
length origin= 157; length generated = 20;
This is unlabeled data 388!
This is unlabeled data 389!
length=182; hm_slices=20;m=9; b=1;e=1
length origin= 182; length generated = 20;
length=160; hm_slices=20;m=8; b=0;e=0
length origin= 160; length generated = 20;
length=131; hm_slices=20;m=6; b=5;e=6
length origin= 131; length generated = 20;
length=123; hm_slices=20;m=6; b=1;e=2
length origin= 123; length generated = 20;
length=323; hm_slices=20;m=16; b=1;e=2
length origin= 323; length generated = 20;
This is unlabeled data 395!
length=254; hm_slices=20;m=12; b=7;e=7
length origin= 254; length generated = 20;
length=128; hm_slices=20;m=6; b=4;e=4
length origin= 128; length generated = 20;
This is unlabeled data 398!
length=167; hm_slices=20;m=8; b=3;e=4
length origin= 167; length generated = 20;
400
length=236; hm_slices=20;m=11; b=8;e=8
length origin= 236; length generated = 20;
length=120; hm_slices=20;m=6; b=0;e=0
length origin= 120; length generated = 20;
length=155; hm_slices=20;m=7; b=7;e=8
length origin= 155; length generated = 20;
length=171; hm_slices=20;m=8; b=5;e=6
length origin= 171; length generated = 20;
This is unlabeled data 404!
length=194; hm_slices=20;m=9; b=7;e=7
length origin= 194; length generated = 20;
length=185; hm_slices=20;m=9; b=2;e=3
length origin= 185; length generated = 20;
length=154; hm_slices=20;m=7; b=7;e=7
length origin= 154; length generated = 20;
length=216; hm_slices=20;m=10; b=8;e=8
length origin= 216; length generated = 20;
length=232; hm_slices=20;m=11; b=6;e=6
length origin= 232; length generated = 20;
length=163; hm_slices=20;m=8; b=1;e=2
length origin= 163; length generated = 20;
length=159; hm_slices=20;m=7; b=9;e=10
length origin= 159; length generated = 20;
length=169; hm_slices=20;m=8; b=4;e=5
length origin= 169; length generated = 20;
length=146; hm_slices=20;m=7; b=3;e=3
length origin= 146; length generated = 20;
length=176; hm_slices=20;m=8; b=8;e=8
length origin= 176; length generated = 20;
length=161; hm_slices=20;m=8; b=0;e=1
length origin= 161; length generated = 20;
length=172; hm_slices=20;m=8; b=6;e=6
length origin= 172; length generated = 20;
length=109; hm_slices=20;m=5; b=4;e=5
length origin= 109; length generated = 20;
This is unlabeled data 418!
length=151; hm_slices=20;m=7; b=5;e=6
length origin= 151; length generated = 20;
length=148; hm_slices=20;m=7; b=4;e=4
length origin= 148; length generated = 20;
length=191; hm_slices=20;m=9; b=5;e=6
length origin= 191; length generated = 20;
This is unlabeled data 422!
length=167; hm_slices=20;m=8; b=3;e=4
length origin= 167; length generated = 20;
length=133; hm_slices=20;m=6; b=6;e=7
length origin= 133; length generated = 20;
length=176; hm_slices=20;m=8; b=8;e=8
length origin= 176; length generated = 20;
length=176; hm_slices=20;m=8; b=8;e=8
length origin= 176; length generated = 20;
This is unlabeled data 427!
length=169; hm_slices=20;m=8; b=4;e=5
length origin= 169; length generated = 20;
length=154; hm_slices=20;m=7; b=7;e=7
length origin= 154; length generated = 20;
length=168; hm_slices=20;m=8; b=4;e=4
length origin= 168; length generated = 20;
length=176; hm_slices=20;m=8; b=8;e=8
length origin= 176; length generated = 20;
This is unlabeled data 432!
length=140; hm_slices=20;m=7; b=0;e=0
length origin= 140; length generated = 20;
length=206; hm_slices=20;m=10; b=3;e=3
length origin= 206; length generated = 20;
length=117; hm_slices=20;m=5; b=8;e=9
length origin= 117; length generated = 20;
length=134; hm_slices=20;m=6; b=7;e=7
length origin= 134; length generated = 20;
length=139; hm_slices=20;m=6; b=9;e=10
length origin= 139; length generated = 20;
length=125; hm_slices=20;m=6; b=2;e=3
length origin= 125; length generated = 20;
This is unlabeled data 439!
length=145; hm_slices=20;m=7; b=2;e=3
length origin= 145; length generated = 20;
length=163; hm_slices=20;m=8; b=1;e=2
length origin= 163; length generated = 20;
length=195; hm_slices=20;m=9; b=7;e=8
length origin= 195; length generated = 20;
length=167; hm_slices=20;m=8; b=3;e=4
length origin= 167; length generated = 20;

length=155; hm_slices=20;m=7; b=7;e=8
length origin= 155; length generated = 20;
length=149; hm_slices=20;m=7; b=4;e=5
length origin= 149; length generated = 20;
This is unlabeled data 446!
length=181; hm_slices=20;m=9; b=0;e=1
length origin= 181; length generated = 20;
length=112; hm_slices=20;m=5; b=6;e=6
length origin= 112; length generated = 20;
length=135; hm_slices=20;m=6; b=7;e=8
length origin= 135; length generated = 20;
This is unlabeled data 450!
length=163; hm_slices=20;m=8; b=1;e=2
length origin= 163; length generated = 20;
length=121; hm_slices=20;m=6; b=0;e=1
length origin= 121; length generated = 20;
length=184; hm_slices=20;m=9; b=2;e=2
length origin= 184; length generated = 20;
length=170; hm_slices=20;m=8; b=5;e=5
length origin= 170; length generated = 20;
length=165; hm_slices=20;m=8; b=2;e=3
length origin= 165; length generated = 20;
length=459; hm_slices=20;m=22; b=9;e=10
length origin= 459; length generated = 20;
length=113; hm_slices=20;m=5; b=6;e=7
length origin= 113; length generated = 20;
length=262; hm_slices=20;m=13; b=1;e=1
length origin= 262; length generated = 20;
length=237; hm_slices=20;m=11; b=8;e=9
length origin= 237; length generated = 20;
length=293; hm_slices=20;m=14; b=6;e=7
length origin= 293; length generated = 20;
This is unlabeled data 461!
This is unlabeled data 462!
length=135; hm_slices=20;m=6; b=7;e=8
length origin= 135; length generated = 20;
This is unlabeled data 464!
length=126; hm_slices=20;m=6; b=3;e=3
length origin= 126; length generated = 20;
This is unlabeled data 466!
length=202; hm_slices=20;m=10; b=1;e=1
length origin= 202; length generated = 20;
length=241; hm_slices=20;m=12; b=0;e=1
length origin= 241; length generated = 20;
length=189; hm_slices=20;m=9; b=4;e=5
length origin= 189; length generated = 20;
length=154; hm_slices=20;m=7; b=7;e=7
length origin= 154; length generated = 20;
length=144; hm_slices=20;m=7; b=2;e=2
length origin= 144; length generated = 20;
length=158; hm_slices=20;m=7; b=9;e=9
length origin= 158; length generated = 20;
length=221; hm_slices=20;m=11; b=0;e=1
length origin= 221; length generated = 20;
length=126; hm_slices=20;m=6; b=3;e=3
length origin= 126; length generated = 20;
This is unlabeled data 475!
length=183; hm_slices=20;m=9; b=1;e=2
length origin= 183; length generated = 20;
length=105; hm_slices=20;m=5; b=2;e=3
length origin= 105; length generated = 20;
This is unlabeled data 478!
length=143; hm_slices=20;m=7; b=1;e=2
length origin= 143; length generated = 20;
length=505; hm_slices=20;m=25; b=2;e=3
length origin= 505; length generated = 20;
This is unlabeled data 481!
length=152; hm_slices=20;m=7; b=6;e=6
length origin= 152; length generated = 20;
length=113; hm_slices=20;m=5; b=6;e=7
length origin= 113; length generated = 20;
length=272; hm_slices=20;m=13; b=6;e=6
length origin= 272; length generated = 20;
length=201; hm_slices=20;m=10; b=0;e=1
length origin= 201; length generated = 20;
This is unlabeled data 486!
length=130; hm_slices=20;m=6; b=5;e=5
length origin= 130; length generated = 20;
length=142; hm_slices=20;m=7; b=1;e=1
length origin= 142; length generated = 20;
length=214; hm_slices=20;m=10; b=7;e=7
length origin= 214; length generated = 20;
length=161; hm_slices=20;m=8; b=0;e=1
length origin= 161; length generated = 20;
length=265; hm_slices=20;m=13; b=2;e=3
length origin= 265; length generated = 20;
length=155; hm_slices=20;m=7; b=7;e=8
length origin= 155; length generated = 20;
length=167; hm_slices=20;m=8; b=3;e=4
length origin= 167; length generated = 20;
length=184; hm_slices=20;m=9; b=2;e=2
length origin= 184; length generated = 20;
This is unlabeled data 495!
length=171; hm_slices=20;m=8; b=5;e=6
length origin= 171; length generated = 20;
This is unlabeled data 497!
length=175; hm_slices=20;m=8; b=7;e=8
length origin= 175; length generated = 20;
length=142; hm_slices=20;m=7; b=1;e=1
length origin= 142; length generated = 20;
500
length=167; hm_slices=20;m=8; b=3;e=4
length origin= 167; length generated = 20;
length=160; hm_slices=20;m=8; b=0;e=0
length origin= 160; length generated = 20;
length=134; hm_slices=20;m=6; b=7;e=7
length origin= 134; length generated = 20;
length=121; hm_slices=20;m=6; b=0;e=1
length origin= 121; length generated = 20;
length=130; hm_slices=20;m=6; b=5;e=5
length origin= 130; length generated = 20;
length=199; hm_slices=20;m=9; b=9;e=10
length origin= 199; length generated = 20;
length=208; hm_slices=20;m=10; b=4;e=4
length origin= 208; length generated = 20;
length=159; hm_slices=20;m=7; b=9;e=10
length origin= 159; length generated = 20;
length=118; hm_slices=20;m=5; b=9;e=9
length origin= 118; length generated = 20;
length=165; hm_slices=20;m=8; b=2;e=3
length origin= 165; length generated = 20;
length=162; hm_slices=20;m=8; b=1;e=1
length origin= 162; length generated = 20;
length=123; hm_slices=20;m=6; b=1;e=2
length origin= 123; length generated = 20;
This is unlabeled data 512!
length=147; hm_slices=20;m=7; b=3;e=4
length origin= 147; length generated = 20;
length=164; hm_slices=20;m=8; b=2;e=2
length origin= 164; length generated = 20;
length=184; hm_slices=20;m=9; b=2;e=2
length origin= 184; length generated = 20;
This is unlabeled data 516!
length=196; hm_slices=20;m=9; b=8;e=8
length origin= 196; length generated = 20;
length=190; hm_slices=20;m=9; b=5;e=5
length origin= 190; length generated = 20;
length=149; hm_slices=20;m=7; b=4;e=5
length origin= 149; length generated = 20;
length=121; hm_slices=20;m=6; b=0;e=1
length origin= 121; length generated = 20;
length=189; hm_slices=20;m=9; b=4;e=5
length origin= 189; length generated = 20;
length=159; hm_slices=20;m=7; b=9;e=10
length origin= 159; length generated = 20;
length=239; hm_slices=20;m=11; b=9;e=10
length origin= 239; length generated = 20;
length=151; hm_slices=20;m=7; b=5;e=6
length origin= 151; length generated = 20;
length=172; hm_slices=20;m=8; b=6;e=6
length origin= 172; length generated = 20;
This is unlabeled data 526!
length=150; hm_slices=20;m=7; b=5;e=5
length origin= 150; length generated = 20;
length=158; hm_slices=20;m=7; b=9;e=9
length origin= 158; length generated = 20;
length=124; hm_slices=20;m=6; b=2;e=2
length origin= 124; length generated = 20;
length=153; hm_slices=20;m=7; b=6;e=7
length origin= 153; length generated = 20;
length=142; hm_slices=20;m=7; b=1;e=1
length origin= 142; length generated = 20;
This is unlabeled data 532!
length=155; hm_slices=20;m=7; b=7;e=8
length origin= 155; length generated = 20;
length=258; hm_slices=20;m=12; b=9;e=9
length origin= 258; length generated = 20;
This is unlabeled data 535!
length=116; hm_slices=20;m=5; b=8;e=8
length origin= 116; length generated = 20;
length=159; hm_slices=20;m=7; b=9;e=10
length origin= 159; length generated = 20;
length=427; hm_slices=20;m=21; b=3;e=4
length origin= 427; length generated = 20;
length=141; hm_slices=20;m=7; b=0;e=1
length origin= 141; length generated = 20;
length=128; hm_slices=20;m=6; b=4;e=4
length origin= 128; length generated = 20;
length=135; hm_slices=20;m=6; b=7;e=8
length origin= 135; length generated = 20;
length=309; hm_slices=20;m=15; b=4;e=5
length origin= 309; length generated = 20;
length=260; hm_slices=20;m=13; b=0;e=0
length origin= 260; length generated = 20;
length=194; hm_slices=20;m=9; b=7;e=7
length origin= 194; length generated = 20;
length=146; hm_slices=20;m=7; b=3;e=3
length origin= 146; length generated = 20;
length=118; hm_slices=20;m=5; b=9;e=9
length origin= 118; length generated = 20;
length=131; hm_slices=20;m=6; b=5;e=6
length origin= 131; length generated = 20;
length=178; hm_slices=20;m=8; b=9;e=9
length origin= 178; length generated = 20;
length=166; hm_slices=20;m=8; b=3;e=3
length origin= 166; length generated = 20;
length=162; hm_slices=20;m=8; b=1;e=1
length origin= 162; length generated = 20;
length=158; hm_slices=20;m=7; b=9;e=9
length origin= 158; length generated = 20;
This is unlabeled data 552!
length=147; hm_slices=20;m=7; b=3;e=4
length origin= 147; length generated = 20;
length=133; hm_slices=20;m=6; b=6;e=7
length origin= 133; length generated = 20;
length=239; hm_slices=20;m=11; b=9;e=10
length origin= 239; length generated = 20;
This is unlabeled data 556!
length=250; hm_slices=20;m=12; b=5;e=5
length origin= 250; length generated = 20;

length=130; hm_slices=20;m=6; b=5;e=5
length origin= 130; length generated = 20;
length=244; hm_slices=20;m=12; b=2;e=2
length origin= 244; length generated = 20;
length=286; hm_slices=20;m=14; b=3;e=3
length origin= 286; length generated = 20;
length=107; hm_slices=20;m=5; b=3;e=4
length origin= 107; length generated = 20;
length=379; hm_slices=20;m=18; b=9;e=10
length origin= 379; length generated = 20;
length=243; hm_slices=20;m=12; b=1;e=2
length origin= 243; length generated = 20;
length=177; hm_slices=20;m=8; b=8;e=9
length origin= 177; length generated = 20;
length=195; hm_slices=20;m=9; b=7;e=8
length origin= 195; length generated = 20;
length=166; hm_slices=20;m=8; b=3;e=3
length origin= 166; length generated = 20;
length=150; hm_slices=20;m=7; b=5;e=5
length origin= 150; length generated = 20;
length=214; hm_slices=20;m=10; b=7;e=7
length origin= 214; length generated = 20;
length=172; hm_slices=20;m=8; b=6;e=6
length origin= 172; length generated = 20;
length=119; hm_slices=20;m=5; b=9;e=10
length origin= 119; length generated = 20;
length=157; hm_slices=20;m=7; b=8;e=9
length origin= 157; length generated = 20;
length=176; hm_slices=20;m=8; b=8;e=8
length origin= 176; length generated = 20;
length=171; hm_slices=20;m=8; b=5;e=6
length origin= 171; length generated = 20;
length=157; hm_slices=20;m=7; b=8;e=9
length origin= 157; length generated = 20;
length=189; hm_slices=20;m=9; b=4;e=5
length origin= 189; length generated = 20;
length=116; hm_slices=20;m=5; b=8;e=8
length origin= 116; length generated = 20;
length=136; hm_slices=20;m=6; b=8;e=8
length origin= 136; length generated = 20;
length=119; hm_slices=20;m=5; b=9;e=10
length origin= 119; length generated = 20;
This is unlabeled data 579!
length=215; hm_slices=20;m=10; b=7;e=8
length origin= 215; length generated = 20;
length=187; hm_slices=20;m=9; b=3;e=4
length origin= 187; length generated = 20;
length=154; hm_slices=20;m=7; b=7;e=7
length origin= 154; length generated = 20;
length=110; hm_slices=20;m=5; b=5;e=5
length origin= 110; length generated = 20;
length=120; hm_slices=20;m=6; b=0;e=0
length origin= 120; length generated = 20;
length=342; hm_slices=20;m=17; b=1;e=1
length origin= 342; length generated = 20;
length=140; hm_slices=20;m=7; b=0;e=0
length origin= 140; length generated = 20;
length=313; hm_slices=20;m=15; b=6;e=7
length origin= 313; length generated = 20;
This is unlabeled data 588!
length=226; hm_slices=20;m=11; b=3;e=3
length origin= 226; length generated = 20;
length=181; hm_slices=20;m=9; b=0;e=1
length origin= 181; length generated = 20;
length=141; hm_slices=20;m=7; b=0;e=1
length origin= 141; length generated = 20;
This is unlabeled data 592!
length=157; hm_slices=20;m=7; b=8;e=9
length origin= 157; length generated = 20;
length=142; hm_slices=20;m=7; b=1;e=1
length origin= 142; length generated = 20;
length=177; hm_slices=20;m=8; b=8;e=9
length origin= 177; length generated = 20;
length=122; hm_slices=20;m=6; b=1;e=1
length origin= 122; length generated = 20;
This is unlabeled data 597!
length=142; hm_slices=20;m=7; b=1;e=1
length origin= 142; length generated = 20;
length=178; hm_slices=20;m=8; b=9;e=9
length origin= 178; length generated = 20;
600
length=141; hm_slices=20;m=7; b=0;e=1
length origin= 141; length generated = 20;
length=111; hm_slices=20;m=5; b=5;e=6
length origin= 111; length generated = 20;
length=170; hm_slices=20;m=8; b=5;e=5
length origin= 170; length generated = 20;
length=171; hm_slices=20;m=8; b=5;e=6
length origin= 171; length generated = 20;
length=174; hm_slices=20;m=8; b=7;e=7
length origin= 174; length generated = 20;
length=215; hm_slices=20;m=10; b=7;e=8
length origin= 215; length generated = 20;
length=123; hm_slices=20;m=6; b=1;e=2
length origin= 123; length generated = 20;
This is unlabeled data 607!
length=154; hm_slices=20;m=7; b=7;e=7
length origin= 154; length generated = 20;
length=134; hm_slices=20;m=6; b=7;e=7
length origin= 134; length generated = 20;
length=277; hm_slices=20;m=13; b=8;e=9
length origin= 277; length generated = 20;
length=130; hm_slices=20;m=6; b=5;e=5
length origin= 130; length generated = 20;
length=153; hm_slices=20;m=7; b=6;e=7
length origin= 153; length generated = 20;
length=136; hm_slices=20;m=6; b=8;e=8
length origin= 136; length generated = 20;
length=138; hm_slices=20;m=6; b=9;e=9
length origin= 138; length generated = 20;
length=166; hm_slices=20;m=8; b=3;e=3
length origin= 166; length generated = 20;
length=114; hm_slices=20;m=5; b=7;e=7
length origin= 114; length generated = 20;
length=250; hm_slices=20;m=12; b=5;e=5
length origin= 250; length generated = 20;
length=149; hm_slices=20;m=7; b=4;e=5
length origin= 149; length generated = 20;
length=229; hm_slices=20;m=11; b=4;e=5
length origin= 229; length generated = 20;
length=167; hm_slices=20;m=8; b=3;e=4
length origin= 167; length generated = 20;
length=159; hm_slices=20;m=7; b=9;e=10
length origin= 159; length generated = 20;
length=123; hm_slices=20;m=6; b=1;e=2
length origin= 123; length generated = 20;
length=220; hm_slices=20;m=11; b=0;e=0
length origin= 220; length generated = 20;
length=249; hm_slices=20;m=12; b=4;e=5
length origin= 249; length generated = 20;
length=120; hm_slices=20;m=6; b=0;e=0
length origin= 120; length generated = 20;
length=136; hm_slices=20;m=6; b=8;e=8
length origin= 136; length generated = 20;
length=487; hm_slices=20;m=24; b=3;e=4
length origin= 487; length generated = 20;
length=123; hm_slices=20;m=6; b=1;e=2
length origin= 123; length generated = 20;
length=457; hm_slices=20;m=22; b=8;e=9
length origin= 457; length generated = 20;
length=128; hm_slices=20;m=6; b=4;e=4
length origin= 128; length generated = 20;
length=165; hm_slices=20;m=8; b=2;e=3
length origin= 165; length generated = 20;
length=145; hm_slices=20;m=7; b=2;e=3
length origin= 145; length generated = 20;
length=168; hm_slices=20;m=8; b=4;e=4
length origin= 168; length generated = 20;
length=174; hm_slices=20;m=8; b=7;e=7
length origin= 174; length generated = 20;
This is unlabeled data 635!
length=162; hm_slices=20;m=8; b=1;e=1
length origin= 162; length generated = 20;
length=114; hm_slices=20;m=5; b=7;e=7
length origin= 114; length generated = 20;
length=122; hm_slices=20;m=6; b=1;e=1
length origin= 122; length generated = 20;
This is unlabeled data 639!
This is unlabeled data 640!
length=305; hm_slices=20;m=15; b=2;e=3
length origin= 305; length generated = 20;
length=163; hm_slices=20;m=8; b=1;e=2
length origin= 163; length generated = 20;
length=282; hm_slices=20;m=14; b=1;e=1
length origin= 282; length generated = 20;
length=241; hm_slices=20;m=12; b=0;e=1
length origin= 241; length generated = 20;
length=130; hm_slices=20;m=6; b=5;e=5
length origin= 130; length generated = 20;
length=135; hm_slices=20;m=6; b=7;e=8
length origin= 135; length generated = 20;
length=253; hm_slices=20;m=12; b=6;e=7
length origin= 253; length generated = 20;
length=126; hm_slices=20;m=6; b=3;e=3
length origin= 126; length generated = 20;
This is unlabeled data 649!
length=232; hm_slices=20;m=11; b=6;e=6
length origin= 232; length generated = 20;
length=149; hm_slices=20;m=7; b=4;e=5
length origin= 149; length generated = 20;
This is unlabeled data 652!
length=168; hm_slices=20;m=8; b=4;e=4
length origin= 168; length generated = 20;
length=139; hm_slices=20;m=6; b=9;e=10
length origin= 139; length generated = 20;
length=209; hm_slices=20;m=10; b=4;e=5
length origin= 209; length generated = 20;
length=460; hm_slices=20;m=23; b=0;e=0
length origin= 460; length generated = 20;
length=156; hm_slices=20;m=7; b=8;e=8
length origin= 156; length generated = 20;
length=191; hm_slices=20;m=9; b=5;e=6
length origin= 191; length generated = 20;
length=145; hm_slices=20;m=7; b=2;e=3
length origin= 145; length generated = 20;
length=241; hm_slices=20;m=12; b=0;e=1
length origin= 241; length generated = 20;
length=163; hm_slices=20;m=8; b=1;e=2
length origin= 163; length generated = 20;
length=193; hm_slices=20;m=9; b=6;e=7
length origin= 193; length generated = 20;
This is unlabeled data 663!
length=164; hm_slices=20;m=8; b=2;e=2
length origin= 164; length generated = 20;
length=133; hm_slices=20;m=6; b=6;e=7
length origin= 133; length generated = 20;

length=253; hm_slices=20;m=12; b=6;e=7
length origin= 253; length generated = 20;
length=167; hm_slices=20;m=8; b=3;e=4
length origin= 167; length generated = 20;
length=114; hm_slices=20;m=5; b=7;e=7
length origin= 114; length generated = 20;
length=171; hm_slices=20;m=8; b=5;e=6
length origin= 171; length generated = 20;
length=165; hm_slices=20;m=8; b=2;e=3
length origin= 165; length generated = 20;
length=188; hm_slices=20;m=9; b=4;e=4
length origin= 188; length generated = 20;
length=121; hm_slices=20;m=6; b=0;e=1
length origin= 121; length generated = 20;
length=151; hm_slices=20;m=7; b=5;e=6
length origin= 151; length generated = 20;
length=122; hm_slices=20;m=6; b=1;e=1
length origin= 122; length generated = 20;
length=159; hm_slices=20;m=7; b=9;e=10
length origin= 159; length generated = 20;
length=142; hm_slices=20;m=7; b=1;e=1
length origin= 142; length generated = 20;
length=258; hm_slices=20;m=12; b=9;e=9
length origin= 258; length generated = 20;
length=167; hm_slices=20;m=8; b=3;e=4
length origin= 167; length generated = 20;
length=243; hm_slices=20;m=12; b=1;e=2
length origin= 243; length generated = 20;
length=198; hm_slices=20;m=9; b=9;e=9
length origin= 198; length generated = 20;
length=258; hm_slices=20;m=12; b=9;e=9
length origin= 258; length generated = 20;
length=154; hm_slices=20;m=7; b=7;e=7
length origin= 154; length generated = 20;
length=173; hm_slices=20;m=8; b=6;e=7
length origin= 173; length generated = 20;
length=184; hm_slices=20;m=9; b=2;e=2
length origin= 184; length generated = 20;
length=143; hm_slices=20;m=7; b=1;e=2
length origin= 143; length generated = 20;
length=151; hm_slices=20;m=7; b=5;e=6
length origin= 151; length generated = 20;
length=177; hm_slices=20;m=8; b=8;e=9
length origin= 177; length generated = 20;
length=265; hm_slices=20;m=13; b=2;e=3
length origin= 265; length generated = 20;
length=189; hm_slices=20;m=9; b=4;e=5
length origin= 189; length generated = 20;
length=177; hm_slices=20;m=8; b=8;e=9
length origin= 177; length generated = 20;
length=146; hm_slices=20;m=7; b=3;e=3
length origin= 146; length generated = 20;
This is unlabeled data 692!
length=156; hm_slices=20;m=7; b=8;e=8
length origin= 156; length generated = 20;
length=151; hm_slices=20;m=7; b=5;e=6
length origin= 151; length generated = 20;
length=123; hm_slices=20;m=6; b=1;e=2
length origin= 123; length generated = 20;
length=114; hm_slices=20;m=5; b=7;e=7
length origin= 114; length generated = 20;
length=140; hm_slices=20;m=7; b=0;e=0
length origin= 140; length generated = 20;
length=127; hm_slices=20;m=6; b=3;e=4
length origin= 127; length generated = 20;
length=141; hm_slices=20;m=7; b=0;e=1
length origin= 141; length generated = 20;
700
length=121; hm_slices=20;m=6; b=0;e=1
length origin= 121; length generated = 20;
length=144; hm_slices=20;m=7; b=2;e=2
length origin= 144; length generated = 20;
length=138; hm_slices=20;m=6; b=9;e=9
length origin= 138; length generated = 20;
length=178; hm_slices=20;m=8; b=9;e=9
length origin= 178; length generated = 20;
length=383; hm_slices=20;m=19; b=1;e=2
length origin= 383; length generated = 20;
length=110; hm_slices=20;m=5; b=5;e=5
length origin= 110; length generated = 20;
length=146; hm_slices=20;m=7; b=3;e=3
length origin= 146; length generated = 20;
length=178; hm_slices=20;m=8; b=9;e=9
length origin= 178; length generated = 20;
length=248; hm_slices=20;m=12; b=4;e=4
length origin= 248; length generated = 20;
length=255; hm_slices=20;m=12; b=7;e=8
length origin= 255; length generated = 20;
length=142; hm_slices=20;m=7; b=1;e=1
length origin= 142; length generated = 20;
length=150; hm_slices=20;m=7; b=5;e=5
length origin= 150; length generated = 20;
length=206; hm_slices=20;m=10; b=3;e=3
length origin= 206; length generated = 20;
length=179; hm_slices=20;m=8; b=9;e=10
length origin= 179; length generated = 20;
length=171; hm_slices=20;m=8; b=5;e=6
length origin= 171; length generated = 20;
length=134; hm_slices=20;m=6; b=7;e=7
length origin= 134; length generated = 20;
length=127; hm_slices=20;m=6; b=3;e=4
length origin= 127; length generated = 20;
length=219; hm_slices=20;m=10; b=9;e=10
length origin= 219; length generated = 20;
length=334; hm_slices=20;m=16; b=7;e=7
length origin= 334; length generated = 20;
length=180; hm_slices=20;m=9; b=0;e=0
length origin= 180; length generated = 20;
length=300; hm_slices=20;m=15; b=0;e=0
length origin= 300; length generated = 20;
length=170; hm_slices=20;m=8; b=5;e=5
length origin= 170; length generated = 20;
length=225; hm_slices=20;m=11; b=2;e=3
length origin= 225; length generated = 20;
length=331; hm_slices=20;m=16; b=5;e=6
length origin= 331; length generated = 20;
length=164; hm_slices=20;m=8; b=2;e=2
length origin= 164; length generated = 20;
length=195; hm_slices=20;m=9; b=7;e=8
length origin= 195; length generated = 20;
length=169; hm_slices=20;m=8; b=4;e=5
length origin= 169; length generated = 20;
length=161; hm_slices=20;m=8; b=0;e=1
length origin= 161; length generated = 20;
length=157; hm_slices=20;m=7; b=8;e=9
length origin= 157; length generated = 20;
length=131; hm_slices=20;m=6; b=5;e=6
length origin= 131; length generated = 20;
This is unlabeled data 730!
length=197; hm_slices=20;m=9; b=8;e=9
length origin= 197; length generated = 20;
length=222; hm_slices=20;m=11; b=1;e=1
length origin= 222; length generated = 20;
length=157; hm_slices=20;m=7; b=8;e=9
length origin= 157; length generated = 20;
length=227; hm_slices=20;m=11; b=3;e=4
length origin= 227; length generated = 20;
length=171; hm_slices=20;m=8; b=5;e=6
length origin= 171; length generated = 20;
length=134; hm_slices=20;m=6; b=7;e=7
length origin= 134; length generated = 20;
length=187; hm_slices=20;m=9; b=3;e=4
length origin= 187; length generated = 20;
length=108; hm_slices=20;m=5; b=4;e=4
length origin= 108; length generated = 20;
length=167; hm_slices=20;m=8; b=3;e=4
length origin= 167; length generated = 20;
length=172; hm_slices=20;m=8; b=6;e=6
length origin= 172; length generated = 20;
length=130; hm_slices=20;m=6; b=5;e=5
length origin= 130; length generated = 20;
length=94; hm_slices=20;m=4; b=7;e=7
length origin= 94; length generated = 20;
length=169; hm_slices=20;m=8; b=4;e=5
length origin= 169; length generated = 20;
length=252; hm_slices=20;m=12; b=6;e=6
length origin= 252; length generated = 20;
length=140; hm_slices=20;m=7; b=0;e=0
length origin= 140; length generated = 20;
length=166; hm_slices=20;m=8; b=3;e=3
length origin= 166; length generated = 20;
length=132; hm_slices=20;m=6; b=6;e=6
length origin= 132; length generated = 20;
length=140; hm_slices=20;m=7; b=0;e=0
length origin= 140; length generated = 20;
length=175; hm_slices=20;m=8; b=7;e=8
length origin= 175; length generated = 20;
length=131; hm_slices=20;m=6; b=5;e=6
length origin= 131; length generated = 20;
length=215; hm_slices=20;m=10; b=7;e=8
length origin= 215; length generated = 20;
length=277; hm_slices=20;m=13; b=8;e=9
length origin= 277; length generated = 20;
length=146; hm_slices=20;m=7; b=3;e=3
length origin= 146; length generated = 20;
length=118; hm_slices=20;m=5; b=9;e=9
length origin= 118; length generated = 20;
length=247; hm_slices=20;m=12; b=3;e=4
length origin= 247; length generated = 20;
length=385; hm_slices=20;m=19; b=2;e=3
length origin= 385; length generated = 20;
length=190; hm_slices=20;m=9; b=5;e=5
length origin= 190; length generated = 20;
length=152; hm_slices=20;m=7; b=6;e=6
length origin= 152; length generated = 20;
length=257; hm_slices=20;m=12; b=8;e=9
length origin= 257; length generated = 20;
length=336; hm_slices=20;m=16; b=8;e=8
length origin= 336; length generated = 20;
length=183; hm_slices=20;m=9; b=1;e=2
length origin= 183; length generated = 20;
length=257; hm_slices=20;m=12; b=8;e=9
length origin= 257; length generated = 20;
length=137; hm_slices=20;m=6; b=8;e=9
length origin= 137; length generated = 20;
length=194; hm_slices=20;m=9; b=7;e=7
length origin= 194; length generated = 20;
length=130; hm_slices=20;m=6; b=5;e=5
length origin= 130; length generated = 20;
length=127; hm_slices=20;m=6; b=3;e=4
length origin= 127; length generated = 20;
length=284; hm_slices=20;m=14; b=2;e=2
length origin= 284; length generated = 20;
length=188; hm_slices=20;m=9; b=4;e=4
length origin= 188; length generated = 20;

length=306; hm_slices=20;m=15; b=3;e=3
length origin= 306; length generated = 20;
length=182; hm_slices=20;m=9; b=1;e=1
length origin= 182; length generated = 20;
length=154; hm_slices=20;m=7; b=7;e=7
length origin= 154; length generated = 20;
length=141; hm_slices=20;m=7; b=0;e=1
length origin= 141; length generated = 20;
length=161; hm_slices=20;m=8; b=0;e=1
length origin= 161; length generated = 20;
This is unlabeled data 774!
length=121; hm_slices=20;m=6; b=0;e=1
length origin= 121; length generated = 20;
length=157; hm_slices=20;m=7; b=8;e=9
length origin= 157; length generated = 20;
length=273; hm_slices=20;m=13; b=6;e=7
length origin= 273; length generated = 20;
length=176; hm_slices=20;m=8; b=8;e=8
length origin= 176; length generated = 20;
length=141; hm_slices=20;m=7; b=0;e=1
length origin= 141; length generated = 20;
This is unlabeled data 780!
length=126; hm_slices=20;m=6; b=3;e=3
length origin= 126; length generated = 20;
length=164; hm_slices=20;m=8; b=2;e=2
length origin= 164; length generated = 20;
length=427; hm_slices=20;m=21; b=3;e=4
length origin= 427; length generated = 20;
length=149; hm_slices=20;m=7; b=4;e=5
length origin= 149; length generated = 20;
length=181; hm_slices=20;m=9; b=0;e=1
length origin= 181; length generated = 20;
length=265; hm_slices=20;m=13; b=2;e=3
length origin= 265; length generated = 20;
length=158; hm_slices=20;m=7; b=9;e=9
length origin= 158; length generated = 20;
length=135; hm_slices=20;m=6; b=7;e=8
length origin= 135; length generated = 20;
length=235; hm_slices=20;m=11; b=7;e=8
length origin= 235; length generated = 20;
length=165; hm_slices=20;m=8; b=2;e=3
length origin= 165; length generated = 20;
length=156; hm_slices=20;m=7; b=8;e=8
length origin= 156; length generated = 20;
length=173; hm_slices=20;m=8; b=6;e=7
length origin= 173; length generated = 20;
length=115; hm_slices=20;m=5; b=7;e=8
length origin= 115; length generated = 20;
This is unlabeled data 794!
This is unlabeled data 795!
length=170; hm_slices=20;m=8; b=5;e=5
length origin= 170; length generated = 20;
length=177; hm_slices=20;m=8; b=8;e=9
length origin= 177; length generated = 20;
length=135; hm_slices=20;m=6; b=7;e=8
length origin= 135; length generated = 20;
length=137; hm_slices=20;m=6; b=8;e=9
length origin= 137; length generated = 20;
800
length=151; hm_slices=20;m=7; b=5;e=6
length origin= 151; length generated = 20;
This is unlabeled data 801!
length=178; hm_slices=20;m=8; b=9;e=9
length origin= 178; length generated = 20;
length=122; hm_slices=20;m=6; b=1;e=1
length origin= 122; length generated = 20;
length=153; hm_slices=20;m=7; b=6;e=7
length origin= 153; length generated = 20;
length=150; hm_slices=20;m=7; b=5;e=5
length origin= 150; length generated = 20;
length=179; hm_slices=20;m=8; b=9;e=10
length origin= 179; length generated = 20;
length=213; hm_slices=20;m=10; b=6;e=7
length origin= 213; length generated = 20;
length=168; hm_slices=20;m=8; b=4;e=4
length origin= 168; length generated = 20;
length=267; hm_slices=20;m=13; b=3;e=4
length origin= 267; length generated = 20;
This is unlabeled data 810!
length=173; hm_slices=20;m=8; b=6;e=7
length origin= 173; length generated = 20;
length=147; hm_slices=20;m=7; b=3;e=4
length origin= 147; length generated = 20;
length=191; hm_slices=20;m=9; b=5;e=6
length origin= 191; length generated = 20;
This is unlabeled data 814!
This is unlabeled data 815!
length=137; hm_slices=20;m=6; b=8;e=9
length origin= 137; length generated = 20;
length=138; hm_slices=20;m=6; b=9;e=9
length origin= 138; length generated = 20;
length=166; hm_slices=20;m=8; b=3;e=3
length origin= 166; length generated = 20;
length=151; hm_slices=20;m=7; b=5;e=6
length origin= 151; length generated = 20;
length=127; hm_slices=20;m=6; b=3;e=4
length origin= 127; length generated = 20;
length=188; hm_slices=20;m=9; b=4;e=4
length origin= 188; length generated = 20;
length=223; hm_slices=20;m=11; b=1;e=2
length origin= 223; length generated = 20;
length=168; hm_slices=20;m=8; b=4;e=4
length origin= 168; length generated = 20;
This is unlabeled data 824!
length=149; hm_slices=20;m=7; b=4;e=5
length origin= 149; length generated = 20;
length=304; hm_slices=20;m=15; b=2;e=2
length origin= 304; length generated = 20;
length=253; hm_slices=20;m=12; b=6;e=7
length origin= 253; length generated = 20;
length=111; hm_slices=20;m=5; b=5;e=6
length origin= 111; length generated = 20;
length=152; hm_slices=20;m=7; b=6;e=6
length origin= 152; length generated = 20;
length=130; hm_slices=20;m=6; b=5;e=5
length origin= 130; length generated = 20;
length=132; hm_slices=20;m=6; b=6;e=6
length origin= 132; length generated = 20;
length=122; hm_slices=20;m=6; b=1;e=1
length origin= 122; length generated = 20;
length=140; hm_slices=20;m=7; b=0;e=0
length origin= 140; length generated = 20;
length=160; hm_slices=20;m=8; b=0;e=0
length origin= 160; length generated = 20;
This is unlabeled data 835!
This is unlabeled data 836!
length=123; hm_slices=20;m=6; b=1;e=2
length origin= 123; length generated = 20;
length=163; hm_slices=20;m=8; b=1;e=2
length origin= 163; length generated = 20;
length=179; hm_slices=20;m=8; b=9;e=10
length origin= 179; length generated = 20;
length=144; hm_slices=20;m=7; b=2;e=2
length origin= 144; length generated = 20;
length=135; hm_slices=20;m=6; b=7;e=8
length origin= 135; length generated = 20;
This is unlabeled data 842!
length=186; hm_slices=20;m=9; b=3;e=3
length origin= 186; length generated = 20;
length=147; hm_slices=20;m=7; b=3;e=4
length origin= 147; length generated = 20;
length=201; hm_slices=20;m=10; b=0;e=1
length origin= 201; length generated = 20;
length=154; hm_slices=20;m=7; b=7;e=7
length origin= 154; length generated = 20;
length=213; hm_slices=20;m=10; b=6;e=7
length origin= 213; length generated = 20;
length=151; hm_slices=20;m=7; b=5;e=6
length origin= 151; length generated = 20;
length=158; hm_slices=20;m=7; b=9;e=9
length origin= 158; length generated = 20;
length=361; hm_slices=20;m=18; b=0;e=1
length origin= 361; length generated = 20;
length=122; hm_slices=20;m=6; b=1;e=1
length origin= 122; length generated = 20;
length=158; hm_slices=20;m=7; b=9;e=9
length origin= 158; length generated = 20;
length=246; hm_slices=20;m=12; b=3;e=3
length origin= 246; length generated = 20;
length=161; hm_slices=20;m=8; b=0;e=1
length origin= 161; length generated = 20;
This is unlabeled data 855!
length=137; hm_slices=20;m=6; b=8;e=9
length origin= 137; length generated = 20;
length=192; hm_slices=20;m=9; b=6;e=6
length origin= 192; length generated = 20;
This is unlabeled data 858!
length=267; hm_slices=20;m=13; b=3;e=4
length origin= 267; length generated = 20;
length=221; hm_slices=20;m=11; b=0;e=1
length origin= 221; length generated = 20;
length=180; hm_slices=20;m=9; b=0;e=0
length origin= 180; length generated = 20;
length=142; hm_slices=20;m=7; b=1;e=1
length origin= 142; length generated = 20;
length=124; hm_slices=20;m=6; b=2;e=2
length origin= 124; length generated = 20;
length=212; hm_slices=20;m=10; b=6;e=6
length origin= 212; length generated = 20;
length=262; hm_slices=20;m=13; b=1;e=1
length origin= 262; length generated = 20;
length=267; hm_slices=20;m=13; b=3;e=4
length origin= 267; length generated = 20;
length=160; hm_slices=20;m=8; b=0;e=0
length origin= 160; length generated = 20;
length=145; hm_slices=20;m=7; b=2;e=3
length origin= 145; length generated = 20;
length=180; hm_slices=20;m=9; b=0;e=0
length origin= 180; length generated = 20;
This is unlabeled data 870!
length=120; hm_slices=20;m=6; b=0;e=0
length origin= 120; length generated = 20;
length=220; hm_slices=20;m=11; b=0;e=0
length origin= 220; length generated = 20;
This is unlabeled data 873!
length=136; hm_slices=20;m=6; b=8;e=8
length origin= 136; length generated = 20;
length=439; hm_slices=20;m=21; b=9;e=10
length origin= 439; length generated = 20;
length=171; hm_slices=20;m=8; b=5;e=6
length origin= 171; length generated = 20;
length=223; hm_slices=20;m=11; b=1;e=2
length origin= 223; length generated = 20;
This is unlabeled data 878!
length=185; hm_slices=20;m=9; b=2;e=3
length origin= 185; length generated = 20;
length=171; hm_slices=20;m=8; b=5;e=6
length origin= 171; length generated = 20;

length=281; hm_slices=20;m=14; b=0;e=1
length origin= 281; length generated = 20;
length=125; hm_slices=20;m=6; b=2;e=3
length origin= 125; length generated = 20;
length=152; hm_slices=20;m=7; b=6;e=6
length origin= 152; length generated = 20;
This is unlabeled data 884!
length=160; hm_slices=20;m=8; b=0;e=0
length origin= 160; length generated = 20;
length=165; hm_slices=20;m=8; b=2;e=3
length origin= 165; length generated = 20;
length=127; hm_slices=20;m=6; b=3;e=4
length origin= 127; length generated = 20;
length=174; hm_slices=20;m=8; b=7;e=7
length origin= 174; length generated = 20;
length=111; hm_slices=20;m=5; b=5;e=6
length origin= 111; length generated = 20;
length=283; hm_slices=20;m=14; b=1;e=2
length origin= 283; length generated = 20;
length=136; hm_slices=20;m=6; b=8;e=8
length origin= 136; length generated = 20;
length=124; hm_slices=20;m=6; b=2;e=2
length origin= 124; length generated = 20;
length=141; hm_slices=20;m=7; b=0;e=1
length origin= 141; length generated = 20;
length=196; hm_slices=20;m=9; b=8;e=8
length origin= 196; length generated = 20;
length=432; hm_slices=20;m=21; b=6;e=6
length origin= 432; length generated = 20;
length=167; hm_slices=20;m=8; b=3;e=4
length origin= 167; length generated = 20;
length=280; hm_slices=20;m=14; b=0;e=0
length origin= 280; length generated = 20;
length=145; hm_slices=20;m=7; b=2;e=3
length origin= 145; length generated = 20;
length=162; hm_slices=20;m=8; b=1;e=1
length origin= 162; length generated = 20;
900
length=138; hm_slices=20;m=6; b=9;e=9
length origin= 138; length generated = 20;
length=137; hm_slices=20;m=6; b=8;e=9
length origin= 137; length generated = 20;
length=178; hm_slices=20;m=8; b=9;e=9
length origin= 178; length generated = 20;
length=179; hm_slices=20;m=8; b=9;e=10
length origin= 179; length generated = 20;
length=151; hm_slices=20;m=7; b=5;e=6
length origin= 151; length generated = 20;
length=157; hm_slices=20;m=7; b=8;e=9
length origin= 157; length generated = 20;
length=117; hm_slices=20;m=5; b=8;e=9
length origin= 117; length generated = 20;
length=182; hm_slices=20;m=9; b=1;e=1
length origin= 182; length generated = 20;
length=174; hm_slices=20;m=8; b=7;e=7
length origin= 174; length generated = 20;
length=109; hm_slices=20;m=5; b=4;e=5
length origin= 109; length generated = 20;
length=149; hm_slices=20;m=7; b=4;e=5
length origin= 149; length generated = 20;
length=138; hm_slices=20;m=6; b=9;e=9
length origin= 138; length generated = 20;
This is unlabeled data 912!
length=164; hm_slices=20;m=8; b=2;e=2
length origin= 164; length generated = 20;
length=177; hm_slices=20;m=8; b=8;e=9
length origin= 177; length generated = 20;
length=158; hm_slices=20;m=7; b=9;e=9
length origin= 158; length generated = 20;
length=162; hm_slices=20;m=8; b=1;e=1
length origin= 162; length generated = 20;
length=151; hm_slices=20;m=7; b=5;e=6
length origin= 151; length generated = 20;
This is unlabeled data 918!
length=147; hm_slices=20;m=7; b=3;e=4
length origin= 147; length generated = 20;
length=124; hm_slices=20;m=6; b=2;e=2
length origin= 124; length generated = 20;
length=118; hm_slices=20;m=5; b=9;e=9
length origin= 118; length generated = 20;
length=149; hm_slices=20;m=7; b=4;e=5
length origin= 149; length generated = 20;
length=125; hm_slices=20;m=6; b=2;e=3
length origin= 125; length generated = 20;
length=153; hm_slices=20;m=7; b=6;e=7
length origin= 153; length generated = 20;
length=166; hm_slices=20;m=8; b=3;e=3
length origin= 166; length generated = 20;
length=141; hm_slices=20;m=7; b=0;e=1
length origin= 141; length generated = 20;
length=154; hm_slices=20;m=7; b=7;e=7
length origin= 154; length generated = 20;
length=266; hm_slices=20;m=13; b=3;e=3
length origin= 266; length generated = 20;
length=115; hm_slices=20;m=5; b=7;e=8
length origin= 115; length generated = 20;
length=178; hm_slices=20;m=8; b=9;e=9
length origin= 178; length generated = 20;
length=162; hm_slices=20;m=8; b=1;e=1
length origin= 162; length generated = 20;
This is unlabeled data 932!
length=129; hm_slices=20;m=6; b=4;e=5
length origin= 129; length generated = 20;
length=152; hm_slices=20;m=7; b=6;e=6
length origin= 152; length generated = 20;
length=154; hm_slices=20;m=7; b=7;e=7
length origin= 154; length generated = 20;
length=225; hm_slices=20;m=11; b=2;e=3
length origin= 225; length generated = 20;
length=114; hm_slices=20;m=5; b=7;e=7
length origin= 114; length generated = 20;
length=148; hm_slices=20;m=7; b=4;e=4
length origin= 148; length generated = 20;
length=189; hm_slices=20;m=9; b=4;e=5
length origin= 189; length generated = 20;
length=148; hm_slices=20;m=7; b=4;e=4
length origin= 148; length generated = 20;
length=150; hm_slices=20;m=7; b=5;e=5
length origin= 150; length generated = 20;
length=141; hm_slices=20;m=7; b=0;e=1
length origin= 141; length generated = 20;
This is unlabeled data 943!
This is unlabeled data 944!
length=171; hm_slices=20;m=8; b=5;e=6
length origin= 171; length generated = 20;
length=149; hm_slices=20;m=7; b=4;e=5
length origin= 149; length generated = 20;
length=177; hm_slices=20;m=8; b=8;e=9
length origin= 177; length generated = 20;
This is unlabeled data 948!
length=120; hm_slices=20;m=6; b=0;e=0
length origin= 120; length generated = 20;
length=238; hm_slices=20;m=11; b=9;e=9
length origin= 238; length generated = 20;
length=150; hm_slices=20;m=7; b=5;e=5
length origin= 150; length generated = 20;
length=176; hm_slices=20;m=8; b=8;e=8
length origin= 176; length generated = 20;
length=183; hm_slices=20;m=9; b=1;e=2
length origin= 183; length generated = 20;
length=207; hm_slices=20;m=10; b=3;e=4
length origin= 207; length generated = 20;
length=160; hm_slices=20;m=8; b=0;e=0
length origin= 160; length generated = 20;
length=154; hm_slices=20;m=7; b=7;e=7
length origin= 154; length generated = 20;
length=158; hm_slices=20;m=7; b=9;e=9
length origin= 158; length generated = 20;
length=350; hm_slices=20;m=17; b=5;e=5
length origin= 350; length generated = 20;
length=116; hm_slices=20;m=5; b=8;e=8
length origin= 116; length generated = 20;
length=124; hm_slices=20;m=6; b=2;e=2
length origin= 124; length generated = 20;
length=201; hm_slices=20;m=10; b=0;e=1
length origin= 201; length generated = 20;
length=268; hm_slices=20;m=13; b=4;e=4
length origin= 268; length generated = 20;
length=183; hm_slices=20;m=9; b=1;e=2
length origin= 183; length generated = 20;
length=234; hm_slices=20;m=11; b=7;e=7
length origin= 234; length generated = 20;
length=223; hm_slices=20;m=11; b=1;e=2
length origin= 223; length generated = 20;
length=301; hm_slices=20;m=15; b=0;e=1
length origin= 301; length generated = 20;
length=254; hm_slices=20;m=12; b=7;e=7
length origin= 254; length generated = 20;
length=146; hm_slices=20;m=7; b=3;e=3
length origin= 146; length generated = 20;
length=280; hm_slices=20;m=14; b=0;e=0
length origin= 280; length generated = 20;
length=111; hm_slices=20;m=5; b=5;e=6
length origin= 111; length generated = 20;
length=169; hm_slices=20;m=8; b=4;e=5
length origin= 169; length generated = 20;
length=174; hm_slices=20;m=8; b=7;e=7
length origin= 174; length generated = 20;
This is unlabeled data 973!
length=220; hm_slices=20;m=11; b=0;e=0
length origin= 220; length generated = 20;
length=146; hm_slices=20;m=7; b=3;e=3
length origin= 146; length generated = 20;
This is unlabeled data 976!
length=244; hm_slices=20;m=12; b=2;e=2
length origin= 244; length generated = 20;
length=203; hm_slices=20;m=10; b=1;e=2
length origin= 203; length generated = 20;
length=376; hm_slices=20;m=18; b=8;e=8
length origin= 376; length generated = 20;
This is unlabeled data 980!
length=147; hm_slices=20;m=7; b=3;e=4
length origin= 147; length generated = 20;
length=153; hm_slices=20;m=7; b=6;e=7
length origin= 153; length generated = 20;
length=132; hm_slices=20;m=6; b=6;e=6
length origin= 132; length generated = 20;
length=151; hm_slices=20;m=7; b=5;e=6
length origin= 151; length generated = 20;
length=165; hm_slices=20;m=8; b=2;e=3
length origin= 165; length generated = 20;
length=171; hm_slices=20;m=8; b=5;e=6
length origin= 171; length generated = 20;
This is unlabeled data 987!
This is unlabeled data 988!
length=442; hm_slices=20;m=22; b=1;e=1
length origin= 442; length generated = 20;

length=184; hm_slices=20;m=9; b=2;e=2
length origin= 184; length generated = 20;
length=142; hm_slices=20;m=7; b=1;e=1
length origin= 142; length generated = 20;
length=145; hm_slices=20;m=7; b=2;e=3
length origin= 145; length generated = 20;
length=205; hm_slices=20;m=10; b=2;e=3
length origin= 205; length generated = 20;
length=182; hm_slices=20;m=9; b=1;e=1
length origin= 182; length generated = 20;
length=158; hm_slices=20;m=7; b=9;e=9
length origin= 158; length generated = 20;
length=120; hm_slices=20;m=6; b=0;e=0
length origin= 120; length generated = 20;
length=104; hm_slices=20;m=5; b=2;e=2
length origin= 104; length generated = 20;
length=173; hm_slices=20;m=8; b=6;e=7
length origin= 173; length generated = 20;
length=114; hm_slices=20;m=5; b=7;e=7
length origin= 114; length generated = 20;
1000
length=108; hm_slices=20;m=5; b=4;e=4
length origin= 108; length generated = 20;
length=155; hm_slices=20;m=7; b=7;e=8
length origin= 155; length generated = 20;
This is unlabeled data 1002!
length=146; hm_slices=20;m=7; b=3;e=3
length origin= 146; length generated = 20;
length=139; hm_slices=20;m=6; b=9;e=10
length origin= 139; length generated = 20;
length=110; hm_slices=20;m=5; b=5;e=5
length origin= 110; length generated = 20;
length=131; hm_slices=20;m=6; b=5;e=6
length origin= 131; length generated = 20;
length=169; hm_slices=20;m=8; b=4;e=5
length origin= 169; length generated = 20;
This is unlabeled data 1008!
length=232; hm_slices=20;m=11; b=6;e=6
length origin= 232; length generated = 20;
length=113; hm_slices=20;m=5; b=6;e=7
length origin= 113; length generated = 20;
length=192; hm_slices=20;m=9; b=6;e=6
length origin= 192; length generated = 20;
length=258; hm_slices=20;m=12; b=9;e=9
length origin= 258; length generated = 20;
length=146; hm_slices=20;m=7; b=3;e=3
length origin= 146; length generated = 20;
length=170; hm_slices=20;m=8; b=5;e=5
length origin= 170; length generated = 20;
This is unlabeled data 1015!
This is unlabeled data 1016!
This is unlabeled data 1017!
length=114; hm_slices=20;m=5; b=7;e=7
length origin= 114; length generated = 20;
length=141; hm_slices=20;m=7; b=0;e=1
length origin= 141; length generated = 20;
length=119; hm_slices=20;m=5; b=9;e=10
length origin= 119; length generated = 20;
length=176; hm_slices=20;m=8; b=8;e=8
length origin= 176; length generated = 20;
length=341; hm_slices=20;m=17; b=0;e=1
length origin= 341; length generated = 20;
length=175; hm_slices=20;m=8; b=7;e=8
length origin= 175; length generated = 20;
length=172; hm_slices=20;m=8; b=6;e=6
length origin= 172; length generated = 20;
length=169; hm_slices=20;m=8; b=4;e=5
length origin= 169; length generated = 20;
length=150; hm_slices=20;m=7; b=5;e=5
length origin= 150; length generated = 20;
length=107; hm_slices=20;m=5; b=3;e=4
length origin= 107; length generated = 20;
length=146; hm_slices=20;m=7; b=3;e=3
length origin= 146; length generated = 20;
length=194; hm_slices=20;m=9; b=7;e=7
length origin= 194; length generated = 20;
length=152; hm_slices=20;m=7; b=6;e=6
length origin= 152; length generated = 20;
length=261; hm_slices=20;m=13; b=0;e=1
length origin= 261; length generated = 20;
length=138; hm_slices=20;m=6; b=9;e=9
length origin= 138; length generated = 20;
length=155; hm_slices=20;m=7; b=7;e=8
length origin= 155; length generated = 20;
length=123; hm_slices=20;m=6; b=1;e=2
length origin= 123; length generated = 20;
This is unlabeled data 1035!
length=128; hm_slices=20;m=6; b=4;e=4
length origin= 128; length generated = 20;
length=318; hm_slices=20;m=15; b=9;e=9
length origin= 318; length generated = 20;
length=216; hm_slices=20;m=10; b=8;e=8
length origin= 216; length generated = 20;
This is unlabeled data 1039!
length=169; hm_slices=20;m=8; b=4;e=5
length origin= 169; length generated = 20;
length=140; hm_slices=20;m=7; b=0;e=0
length origin= 140; length generated = 20;
length=160; hm_slices=20;m=8; b=0;e=0
length origin= 160; length generated = 20;
length=118; hm_slices=20;m=5; b=9;e=9
length origin= 118; length generated = 20;
This is unlabeled data 1044!
length=134; hm_slices=20;m=6; b=7;e=7
length origin= 134; length generated = 20;
length=127; hm_slices=20;m=6; b=3;e=4
length origin= 127; length generated = 20;
length=140; hm_slices=20;m=7; b=0;e=0
length origin= 140; length generated = 20;
length=193; hm_slices=20;m=9; b=6;e=7
length origin= 193; length generated = 20;
length=259; hm_slices=20;m=12; b=9;e=10
length origin= 259; length generated = 20;
length=167; hm_slices=20;m=8; b=3;e=4
length origin= 167; length generated = 20;
length=148; hm_slices=20;m=7; b=4;e=4
length origin= 148; length generated = 20;
length=149; hm_slices=20;m=7; b=4;e=5
length origin= 149; length generated = 20;
length=179; hm_slices=20;m=8; b=9;e=10
length origin= 179; length generated = 20;
length=145; hm_slices=20;m=7; b=2;e=3
length origin= 145; length generated = 20;
length=130; hm_slices=20;m=6; b=5;e=5
length origin= 130; length generated = 20;
length=119; hm_slices=20;m=5; b=9;e=10
length origin= 119; length generated = 20;
length=256; hm_slices=20;m=12; b=8;e=8
length origin= 256; length generated = 20;
length=179; hm_slices=20;m=8; b=9;e=10
length origin= 179; length generated = 20;
length=160; hm_slices=20;m=8; b=0;e=0
length origin= 160; length generated = 20;
length=155; hm_slices=20;m=7; b=7;e=8
length origin= 155; length generated = 20;
length=195; hm_slices=20;m=9; b=7;e=8
length origin= 195; length generated = 20;
length=201; hm_slices=20;m=10; b=0;e=1
length origin= 201; length generated = 20;
length=125; hm_slices=20;m=6; b=2;e=3
length origin= 125; length generated = 20;
length=506; hm_slices=20;m=25; b=3;e=3
length origin= 506; length generated = 20;
length=181; hm_slices=20;m=9; b=0;e=1
length origin= 181; length generated = 20;
This is unlabeled data 1066!
length=168; hm_slices=20;m=8; b=4;e=4
length origin= 168; length generated = 20;
length=265; hm_slices=20;m=13; b=2;e=3
length origin= 265; length generated = 20;
length=133; hm_slices=20;m=6; b=6;e=7
length origin= 133; length generated = 20;
length=176; hm_slices=20;m=8; b=8;e=8
length origin= 176; length generated = 20;
This is unlabeled data 1071!
length=211; hm_slices=20;m=10; b=5;e=6
length origin= 211; length generated = 20;
length=146; hm_slices=20;m=7; b=3;e=3
length origin= 146; length generated = 20;
length=147; hm_slices=20;m=7; b=3;e=4
length origin= 147; length generated = 20;
length=239; hm_slices=20;m=11; b=9;e=10
length origin= 239; length generated = 20;
length=116; hm_slices=20;m=5; b=8;e=8
length origin= 116; length generated = 20;
length=128; hm_slices=20;m=6; b=4;e=4
length origin= 128; length generated = 20;
length=163; hm_slices=20;m=8; b=1;e=2
length origin= 163; length generated = 20;
length=179; hm_slices=20;m=8; b=9;e=10
length origin= 179; length generated = 20;
length=230; hm_slices=20;m=11; b=5;e=5
length origin= 230; length generated = 20;
This is unlabeled data 1081!
This is unlabeled data 1082!
length=202; hm_slices=20;m=10; b=1;e=1
length origin= 202; length generated = 20;
length=128; hm_slices=20;m=6; b=4;e=4
length origin= 128; length generated = 20;
length=150; hm_slices=20;m=7; b=5;e=5
length origin= 150; length generated = 20;
length=143; hm_slices=20;m=7; b=1;e=2
length origin= 143; length generated = 20;
length=150; hm_slices=20;m=7; b=5;e=5
length origin= 150; length generated = 20;
length=263; hm_slices=20;m=13; b=1;e=2
length origin= 263; length generated = 20;
length=131; hm_slices=20;m=6; b=5;e=6
length origin= 131; length generated = 20;
length=114; hm_slices=20;m=5; b=7;e=7
length origin= 114; length generated = 20;
length=169; hm_slices=20;m=8; b=4;e=5
length origin= 169; length generated = 20;
length=120; hm_slices=20;m=6; b=0;e=0
length origin= 120; length generated = 20;
length=257; hm_slices=20;m=12; b=8;e=9
length origin= 257; length generated = 20;
length=152; hm_slices=20;m=7; b=6;e=6
length origin= 152; length generated = 20;
length=183; hm_slices=20;m=9; b=1;e=2
length origin= 183; length generated = 20;
length=135; hm_slices=20;m=6; b=7;e=8
length origin= 135; length generated = 20;
length=260; hm_slices=20;m=13; b=0;e=0
length origin= 260; length generated = 20;
length=136; hm_slices=20;m=6; b=8;e=8
length origin= 136; length generated = 20;

length=160; hm_slices=20;m=8; b=0;e=0
length origin= 160; length generated = 20;
1100
length=123; hm_slices=20;m=6; b=1;e=2
length origin= 123; length generated = 20;
length=152; hm_slices=20;m=7; b=6;e=6
length origin= 152; length generated = 20;
length=208; hm_slices=20;m=10; b=4;e=4
length origin= 208; length generated = 20;
length=157; hm_slices=20;m=7; b=8;e=9
length origin= 157; length generated = 20;
length=256; hm_slices=20;m=12; b=8;e=8
length origin= 256; length generated = 20;
length=130; hm_slices=20;m=6; b=5;e=5
length origin= 130; length generated = 20;
This is unlabeled data 1106!
This is unlabeled data 1107!
length=118; hm_slices=20;m=5; b=9;e=9
length origin= 118; length generated = 20;
length=128; hm_slices=20;m=6; b=4;e=4
length origin= 128; length generated = 20;
length=205; hm_slices=20;m=10; b=2;e=3
length origin= 205; length generated = 20;
This is unlabeled data 1111!
length=162; hm_slices=20;m=8; b=1;e=1
length origin= 162; length generated = 20;
length=269; hm_slices=20;m=13; b=4;e=5
length origin= 269; length generated = 20;
length=178; hm_slices=20;m=8; b=9;e=9
length origin= 178; length generated = 20;
length=132; hm_slices=20;m=6; b=6;e=6
length origin= 132; length generated = 20;
length=132; hm_slices=20;m=6; b=6;e=6
length origin= 132; length generated = 20;
length=282; hm_slices=20;m=14; b=1;e=1
length origin= 282; length generated = 20;
length=157; hm_slices=20;m=7; b=8;e=9
length origin= 157; length generated = 20;
This is unlabeled data 1119!
length=105; hm_slices=20;m=5; b=2;e=3
length origin= 105; length generated = 20;
length=182; hm_slices=20;m=9; b=1;e=1
length origin= 182; length generated = 20;
length=188; hm_slices=20;m=9; b=4;e=4
length origin= 188; length generated = 20;
length=159; hm_slices=20;m=7; b=9;e=10
length origin= 159; length generated = 20;
This is unlabeled data 1124!
length=180; hm_slices=20;m=9; b=0;e=0
length origin= 180; length generated = 20;
length=155; hm_slices=20;m=7; b=7;e=8
length origin= 155; length generated = 20;
length=145; hm_slices=20;m=7; b=2;e=3
length origin= 145; length generated = 20;
length=328; hm_slices=20;m=16; b=4;e=4
length origin= 328; length generated = 20;
length=134; hm_slices=20;m=6; b=7;e=7
length origin= 134; length generated = 20;
This is unlabeled data 1130!
length=184; hm_slices=20;m=9; b=2;e=2
length origin= 184; length generated = 20;
length=146; hm_slices=20;m=7; b=3;e=3
length origin= 146; length generated = 20;
length=123; hm_slices=20;m=6; b=1;e=2
length origin= 123; length generated = 20;
length=171; hm_slices=20;m=8; b=5;e=6
length origin= 171; length generated = 20;
length=163; hm_slices=20;m=8; b=1;e=2
length origin= 163; length generated = 20;
This is unlabeled data 1136!
length=131; hm_slices=20;m=6; b=5;e=6
length origin= 131; length generated = 20;
length=123; hm_slices=20;m=6; b=1;e=2
length origin= 123; length generated = 20;
length=155; hm_slices=20;m=7; b=7;e=8
length origin= 155; length generated = 20;
length=249; hm_slices=20;m=12; b=4;e=5
length origin= 249; length generated = 20;
length=132; hm_slices=20;m=6; b=6;e=6
length origin= 132; length generated = 20;
This is unlabeled data 1142!
length=162; hm_slices=20;m=8; b=1;e=1
length origin= 162; length generated = 20;
length=130; hm_slices=20;m=6; b=5;e=5
length origin= 130; length generated = 20;
length=166; hm_slices=20;m=8; b=3;e=3
length origin= 166; length generated = 20;
length=148; hm_slices=20;m=7; b=4;e=4
length origin= 148; length generated = 20;
length=173; hm_slices=20;m=8; b=6;e=7
length origin= 173; length generated = 20;
length=124; hm_slices=20;m=6; b=2;e=2
length origin= 124; length generated = 20;
This is unlabeled data 1149!
length=124; hm_slices=20;m=6; b=2;e=2
length origin= 124; length generated = 20;
length=173; hm_slices=20;m=8; b=6;e=7
length origin= 173; length generated = 20;
length=251; hm_slices=20;m=12; b=5;e=6
length origin= 251; length generated = 20;
This is unlabeled data 1153!
length=114; hm_slices=20;m=5; b=7;e=7
length origin= 114; length generated = 20;
length=251; hm_slices=20;m=12; b=5;e=6
length origin= 251; length generated = 20;
length=113; hm_slices=20;m=5; b=6;e=7
length origin= 113; length generated = 20;
length=124; hm_slices=20;m=6; b=2;e=2
length origin= 124; length generated = 20;
length=165; hm_slices=20;m=8; b=2;e=3
length origin= 165; length generated = 20;
length=139; hm_slices=20;m=6; b=9;e=10
length origin= 139; length generated = 20;
length=129; hm_slices=20;m=6; b=4;e=5
length origin= 129; length generated = 20;
This is unlabeled data 1161!
length=205; hm_slices=20;m=10; b=2;e=3
length origin= 205; length generated = 20;
length=140; hm_slices=20;m=7; b=0;e=0
length origin= 140; length generated = 20;
This is unlabeled data 1164!
length=167; hm_slices=20;m=8; b=3;e=4
length origin= 167; length generated = 20;
length=120; hm_slices=20;m=6; b=0;e=0
length origin= 120; length generated = 20;
length=177; hm_slices=20;m=8; b=8;e=9
length origin= 177; length generated = 20;
length=142; hm_slices=20;m=7; b=1;e=1
length origin= 142; length generated = 20;
length=265; hm_slices=20;m=13; b=2;e=3
length origin= 265; length generated = 20;
length=163; hm_slices=20;m=8; b=1;e=2
length origin= 163; length generated = 20;
length=144; hm_slices=20;m=7; b=2;e=2
length origin= 144; length generated = 20;
length=248; hm_slices=20;m=12; b=4;e=4
length origin= 248; length generated = 20;
length=236; hm_slices=20;m=11; b=8;e=8
length origin= 236; length generated = 20;
length=175; hm_slices=20;m=8; b=7;e=8
length origin= 175; length generated = 20;
This is unlabeled data 1175!
length=166; hm_slices=20;m=8; b=3;e=3
length origin= 166; length generated = 20;
length=162; hm_slices=20;m=8; b=1;e=1
length origin= 162; length generated = 20;
This is unlabeled data 1178!
length=125; hm_slices=20;m=6; b=2;e=3
length origin= 125; length generated = 20;
length=353; hm_slices=20;m=17; b=6;e=7
length origin= 353; length generated = 20;
length=150; hm_slices=20;m=7; b=5;e=5
length origin= 150; length generated = 20;
length=162; hm_slices=20;m=8; b=1;e=1
length origin= 162; length generated = 20;
length=115; hm_slices=20;m=5; b=7;e=8
length origin= 115; length generated = 20;
length=197; hm_slices=20;m=9; b=8;e=9
length origin= 197; length generated = 20;
length=181; hm_slices=20;m=9; b=0;e=1
length origin= 181; length generated = 20;
length=173; hm_slices=20;m=8; b=6;e=7
length origin= 173; length generated = 20;
length=155; hm_slices=20;m=7; b=7;e=8
length origin= 155; length generated = 20;
length=159; hm_slices=20;m=7; b=9;e=10
length origin= 159; length generated = 20;
length=166; hm_slices=20;m=8; b=3;e=3
length origin= 166; length generated = 20;
length=217; hm_slices=20;m=10; b=8;e=9
length origin= 217; length generated = 20;
length=202; hm_slices=20;m=10; b=1;e=1
length origin= 202; length generated = 20;
length=170; hm_slices=20;m=8; b=5;e=5
length origin= 170; length generated = 20;
length=133; hm_slices=20;m=6; b=6;e=7
length origin= 133; length generated = 20;
length=304; hm_slices=20;m=15; b=2;e=2
length origin= 304; length generated = 20;
length=155; hm_slices=20;m=7; b=7;e=8
length origin= 155; length generated = 20;
This is unlabeled data 1196!
length=132; hm_slices=20;m=6; b=6;e=6
length origin= 132; length generated = 20;
length=114; hm_slices=20;m=5; b=7;e=7
length origin= 114; length generated = 20;
length=116; hm_slices=20;m=5; b=8;e=8
length origin= 116; length generated = 20;
1200
length=172; hm_slices=20;m=8; b=6;e=6
length origin= 172; length generated = 20;
length=147; hm_slices=20;m=7; b=3;e=4
length origin= 147; length generated = 20;
length=186; hm_slices=20;m=9; b=3;e=3
length origin= 186; length generated = 20;
length=189; hm_slices=20;m=9; b=4;e=5
length origin= 189; length generated = 20;
length=149; hm_slices=20;m=7; b=4;e=5
length origin= 149; length generated = 20;
length=130; hm_slices=20;m=6; b=5;e=5
length origin= 130; length generated = 20;
length=156; hm_slices=20;m=7; b=8;e=8
length origin= 156; length generated = 20;
length=158; hm_slices=20;m=7; b=9;e=9
length origin= 158; length generated = 20;
This is unlabeled data 1208!
length=158; hm_slices=20;m=7; b=9;e=9
length origin= 158; length generated = 20;
length=252; hm_slices=20;m=12; b=6;e=6
length origin= 252; length generated = 20;

length=228; hm_slices=20;m=11; b=4;e=4
length origin= 228; length generated = 20;
length=269; hm_slices=20;m=13; b=4;e=5
length origin= 269; length generated = 20;
length=282; hm_slices=20;m=14; b=1;e=1
length origin= 282; length generated = 20;
length=124; hm_slices=20;m=6; b=2;e=2
length origin= 124; length generated = 20;
length=122; hm_slices=20;m=6; b=1;e=1
length origin= 122; length generated = 20;
length=120; hm_slices=20;m=6; b=0;e=0
length origin= 120; length generated = 20;
This is unlabeled data 1217!
length=195; hm_slices=20;m=9; b=7;e=8
length origin= 195; length generated = 20;
length=149; hm_slices=20;m=7; b=4;e=5
length origin= 149; length generated = 20;
length=171; hm_slices=20;m=8; b=5;e=6
length origin= 171; length generated = 20;
length=133; hm_slices=20;m=6; b=6;e=7
length origin= 133; length generated = 20;
length=276; hm_slices=20;m=13; b=8;e=8
length origin= 276; length generated = 20;
length=138; hm_slices=20;m=6; b=9;e=9
length origin= 138; length generated = 20;
length=324; hm_slices=20;m=16; b=2;e=2
length origin= 324; length generated = 20;
This is unlabeled data 1225!
length=366; hm_slices=20;m=18; b=3;e=3
length origin= 366; length generated = 20;
This is unlabeled data 1227!
length=178; hm_slices=20;m=8; b=9;e=9
length origin= 178; length generated = 20;
length=151; hm_slices=20;m=7; b=5;e=6
length origin= 151; length generated = 20;
This is unlabeled data 1230!
length=176; hm_slices=20;m=8; b=8;e=8
length origin= 176; length generated = 20;
length=158; hm_slices=20;m=7; b=9;e=9
length origin= 158; length generated = 20;
length=132; hm_slices=20;m=6; b=6;e=6
length origin= 132; length generated = 20;
length=244; hm_slices=20;m=12; b=2;e=2
length origin= 244; length generated = 20;
length=222; hm_slices=20;m=11; b=1;e=1
length origin= 222; length generated = 20;
length=103; hm_slices=20;m=5; b=1;e=2
length origin= 103; length generated = 20;
length=167; hm_slices=20;m=8; b=3;e=4
length origin= 167; length generated = 20;
length=254; hm_slices=20;m=12; b=7;e=7
length origin= 254; length generated = 20;
length=167; hm_slices=20;m=8; b=3;e=4
length origin= 167; length generated = 20;
length=328; hm_slices=20;m=16; b=4;e=4
length origin= 328; length generated = 20;
length=153; hm_slices=20;m=7; b=6;e=7
length origin= 153; length generated = 20;
length=169; hm_slices=20;m=8; b=4;e=5
length origin= 169; length generated = 20;
length=175; hm_slices=20;m=8; b=7;e=8
length origin= 175; length generated = 20;
length=169; hm_slices=20;m=8; b=4;e=5
length origin= 169; length generated = 20;
length=155; hm_slices=20;m=7; b=7;e=8
length origin= 155; length generated = 20;
length=110; hm_slices=20;m=5; b=5;e=5
length origin= 110; length generated = 20;
length=148; hm_slices=20;m=7; b=4;e=4
length origin= 148; length generated = 20;
length=277; hm_slices=20;m=13; b=8;e=9
length origin= 277; length generated = 20;
This is unlabeled data 1249!
This is unlabeled data 1250!
This is unlabeled data 1251!
length=241; hm_slices=20;m=12; b=0;e=1
length origin= 241; length generated = 20;
This is unlabeled data 1253!
length=169; hm_slices=20;m=8; b=4;e=5
length origin= 169; length generated = 20;
length=264; hm_slices=20;m=13; b=2;e=2
length origin= 264; length generated = 20;
length=245; hm_slices=20;m=12; b=2;e=3
length origin= 245; length generated = 20;
This is unlabeled data 1257!
length=162; hm_slices=20;m=8; b=1;e=1
length origin= 162; length generated = 20;
length=112; hm_slices=20;m=5; b=6;e=6
length origin= 112; length generated = 20;
length=180; hm_slices=20;m=9; b=0;e=0
length origin= 180; length generated = 20;
length=140; hm_slices=20;m=7; b=0;e=0
length origin= 140; length generated = 20;
length=159; hm_slices=20;m=7; b=9;e=10
length origin= 159; length generated = 20;
length=161; hm_slices=20;m=8; b=0;e=1
length origin= 161; length generated = 20;
length=139; hm_slices=20;m=6; b=9;e=10
length origin= 139; length generated = 20;
length=225; hm_slices=20;m=11; b=2;e=3
length origin= 225; length generated = 20;
length=144; hm_slices=20;m=7; b=2;e=2
length origin= 144; length generated = 20;
This is unlabeled data 1267!
length=184; hm_slices=20;m=9; b=2;e=2
length origin= 184; length generated = 20;
length=112; hm_slices=20;m=5; b=6;e=6
length origin= 112; length generated = 20;
length=154; hm_slices=20;m=7; b=7;e=7
length origin= 154; length generated = 20;
length=180; hm_slices=20;m=9; b=0;e=0
length origin= 180; length generated = 20;
length=216; hm_slices=20;m=10; b=8;e=8
length origin= 216; length generated = 20;
length=146; hm_slices=20;m=7; b=3;e=3
length origin= 146; length generated = 20;
length=154; hm_slices=20;m=7; b=7;e=7
length origin= 154; length generated = 20;
length=158; hm_slices=20;m=7; b=9;e=9
length origin= 158; length generated = 20;
length=141; hm_slices=20;m=7; b=0;e=1
length origin= 141; length generated = 20;
length=142; hm_slices=20;m=7; b=1;e=1
length origin= 142; length generated = 20;
length=156; hm_slices=20;m=7; b=8;e=8
length origin= 156; length generated = 20;
length=158; hm_slices=20;m=7; b=9;e=9
length origin= 158; length generated = 20;
length=156; hm_slices=20;m=7; b=8;e=8
length origin= 156; length generated = 20;
length=156; hm_slices=20;m=7; b=8;e=8
length origin= 156; length generated = 20;
length=157; hm_slices=20;m=7; b=8;e=9
length origin= 157; length generated = 20;
This is unlabeled data 1283!
length=117; hm_slices=20;m=5; b=8;e=9
length origin= 117; length generated = 20;
length=157; hm_slices=20;m=7; b=8;e=9
length origin= 157; length generated = 20;
length=136; hm_slices=20;m=6; b=8;e=8
length origin= 136; length generated = 20;
This is unlabeled data 1287!
length=265; hm_slices=20;m=13; b=2;e=3
length origin= 265; length generated = 20;
length=121; hm_slices=20;m=6; b=0;e=1
length origin= 121; length generated = 20;
This is unlabeled data 1290!
length=147; hm_slices=20;m=7; b=3;e=4
length origin= 147; length generated = 20;
length=140; hm_slices=20;m=7; b=0;e=0
length origin= 140; length generated = 20;
length=267; hm_slices=20;m=13; b=3;e=4
length origin= 267; length generated = 20;
length=173; hm_slices=20;m=8; b=6;e=7
length origin= 173; length generated = 20;
length=123; hm_slices=20;m=6; b=1;e=2
length origin= 123; length generated = 20;
length=132; hm_slices=20;m=6; b=6;e=6
length origin= 132; length generated = 20;
length=225; hm_slices=20;m=11; b=2;e=3
length origin= 225; length generated = 20;
length=128; hm_slices=20;m=6; b=4;e=4
length origin= 128; length generated = 20;
length=197; hm_slices=20;m=9; b=8;e=9
length origin= 197; length generated = 20;
1300
length=268; hm_slices=20;m=13; b=4;e=4
length origin= 268; length generated = 20;
length=141; hm_slices=20;m=7; b=0;e=1
length origin= 141; length generated = 20;
length=169; hm_slices=20;m=8; b=4;e=5
length origin= 169; length generated = 20;
length=126; hm_slices=20;m=6; b=3;e=3
length origin= 126; length generated = 20;
length=110; hm_slices=20;m=5; b=5;e=5
length origin= 110; length generated = 20;
length=133; hm_slices=20;m=6; b=6;e=7
length origin= 133; length generated = 20;
length=534; hm_slices=20;m=26; b=7;e=7
length origin= 534; length generated = 20;
This is unlabeled data 1307!
This is unlabeled data 1308!
length=188; hm_slices=20;m=9; b=4;e=4
length origin= 188; length generated = 20;
This is unlabeled data 1310!
length=192; hm_slices=20;m=9; b=6;e=6
length origin= 192; length generated = 20;
length=130; hm_slices=20;m=6; b=5;e=5
length origin= 130; length generated = 20;
length=122; hm_slices=20;m=6; b=1;e=1
length origin= 122; length generated = 20;
length=116; hm_slices=20;m=5; b=8;e=8
length origin= 116; length generated = 20;
length=178; hm_slices=20;m=8; b=9;e=9
length origin= 178; length generated = 20;
length=159; hm_slices=20;m=7; b=9;e=10
length origin= 159; length generated = 20;
This is unlabeled data 1317!
length=217; hm_slices=20;m=10; b=8;e=9
length origin= 217; length generated = 20;
length=176; hm_slices=20;m=8; b=8;e=8
length origin= 176; length generated = 20;
length=120; hm_slices=20;m=6; b=0;e=0
length origin= 120; length generated = 20;
This is unlabeled data 1321!
length=168; hm_slices=20;m=8; b=4;e=4
length origin= 168; length generated = 20;
length=160; hm_slices=20;m=8; b=0;e=0
length origin= 160; length generated = 20;

length=147; hm_slices=20;m=7; b=3;e=4
length origin= 147; length generated = 20;
length=285; hm_slices=20;m=14; b=2;e=3
length origin= 285; length generated = 20;
length=108; hm_slices=20;m=5; b=4;e=4
length origin= 108; length generated = 20;
length=173; hm_slices=20;m=8; b=6;e=7
length origin= 173; length generated = 20;
length=152; hm_slices=20;m=7; b=6;e=6
length origin= 152; length generated = 20;
length=143; hm_slices=20;m=7; b=1;e=2
length origin= 143; length generated = 20;
length=212; hm_slices=20;m=10; b=6;e=6
length origin= 212; length generated = 20;
length=146; hm_slices=20;m=7; b=3;e=3
length origin= 146; length generated = 20;
length=145; hm_slices=20;m=7; b=2;e=3
length origin= 145; length generated = 20;
length=130; hm_slices=20;m=6; b=5;e=5
length origin= 130; length generated = 20;
length=248; hm_slices=20;m=12; b=4;e=4
length origin= 248; length generated = 20;
length=358; hm_slices=20;m=17; b=9;e=9
length origin= 358; length generated = 20;
length=152; hm_slices=20;m=7; b=6;e=6
length origin= 152; length generated = 20;
length=161; hm_slices=20;m=8; b=0;e=1
length origin= 161; length generated = 20;
This is unlabeled data 1338!
length=226; hm_slices=20;m=11; b=3;e=3
length origin= 226; length generated = 20;
length=125; hm_slices=20;m=6; b=2;e=3
length origin= 125; length generated = 20;
length=152; hm_slices=20;m=7; b=6;e=6
length origin= 152; length generated = 20;
length=145; hm_slices=20;m=7; b=2;e=3
length origin= 145; length generated = 20;
This is unlabeled data 1343!
This is unlabeled data 1344!
length=247; hm_slices=20;m=12; b=3;e=4
length origin= 247; length generated = 20;
length=150; hm_slices=20;m=7; b=5;e=5
length origin= 150; length generated = 20;
This is unlabeled data 1347!
length=130; hm_slices=20;m=6; b=5;e=5
length origin= 130; length generated = 20;
length=165; hm_slices=20;m=8; b=2;e=3
length origin= 165; length generated = 20;
length=123; hm_slices=20;m=6; b=1;e=2
length origin= 123; length generated = 20;
length=143; hm_slices=20;m=7; b=1;e=2
length origin= 143; length generated = 20;
length=162; hm_slices=20;m=8; b=1;e=1
length origin= 162; length generated = 20;
length=177; hm_slices=20;m=8; b=8;e=9
length origin= 177; length generated = 20;
length=264; hm_slices=20;m=13; b=2;e=2
length origin= 264; length generated = 20;
length=204; hm_slices=20;m=10; b=2;e=2
length origin= 204; length generated = 20;
length=235; hm_slices=20;m=11; b=7;e=8
length origin= 235; length generated = 20;
This is unlabeled data 1357!
This is unlabeled data 1358!
length=361; hm_slices=20;m=18; b=0;e=1
length origin= 361; length generated = 20;
length=185; hm_slices=20;m=9; b=2;e=3
length origin= 185; length generated = 20;
length=103; hm_slices=20;m=5; b=1;e=2
length origin= 103; length generated = 20;
length=179; hm_slices=20;m=8; b=9;e=10
length origin= 179; length generated = 20;
length=126; hm_slices=20;m=6; b=3;e=3
length origin= 126; length generated = 20;
length=110; hm_slices=20;m=5; b=5;e=5
length origin= 110; length generated = 20;
length=137; hm_slices=20;m=6; b=8;e=9
length origin= 137; length generated = 20;
length=179; hm_slices=20;m=8; b=9;e=10
length origin= 179; length generated = 20;
length=152; hm_slices=20;m=7; b=6;e=6
length origin= 152; length generated = 20;
length=385; hm_slices=20;m=19; b=2;e=3
length origin= 385; length generated = 20;
length=170; hm_slices=20;m=8; b=5;e=5
length origin= 170; length generated = 20;
length=135; hm_slices=20;m=6; b=7;e=8
length origin= 135; length generated = 20;
length=143; hm_slices=20;m=7; b=1;e=2
length origin= 143; length generated = 20;
This is unlabeled data 1372!
length=128; hm_slices=20;m=6; b=4;e=4
length origin= 128; length generated = 20;
length=130; hm_slices=20;m=6; b=5;e=5
length origin= 130; length generated = 20;
length=134; hm_slices=20;m=6; b=7;e=7
length origin= 134; length generated = 20;
length=180; hm_slices=20;m=9; b=0;e=0
length origin= 180; length generated = 20;
length=331; hm_slices=20;m=16; b=5;e=6
length origin= 331; length generated = 20;
length=152; hm_slices=20;m=7; b=6;e=6
length origin= 152; length generated = 20;
length=118; hm_slices=20;m=5; b=9;e=9
length origin= 118; length generated = 20;
length=168; hm_slices=20;m=8; b=4;e=4
length origin= 168; length generated = 20;
length=223; hm_slices=20;m=11; b=1;e=2
length origin= 223; length generated = 20;
This is unlabeled data 1382!
length=145; hm_slices=20;m=7; b=2;e=3
length origin= 145; length generated = 20;
length=149; hm_slices=20;m=7; b=4;e=5
length origin= 149; length generated = 20;
length=145; hm_slices=20;m=7; b=2;e=3
length origin= 145; length generated = 20;
length=154; hm_slices=20;m=7; b=7;e=7
length origin= 154; length generated = 20;
length=259; hm_slices=20;m=12; b=9;e=10
length origin= 259; length generated = 20;
length=233; hm_slices=20;m=11; b=6;e=7
length origin= 233; length generated = 20;
length=203; hm_slices=20;m=10; b=1;e=2
length origin= 203; length generated = 20;
length=166; hm_slices=20;m=8; b=3;e=3
length origin= 166; length generated = 20;
length=177; hm_slices=20;m=8; b=8;e=9
length origin= 177; length generated = 20;
length=525; hm_slices=20;m=26; b=2;e=3
length origin= 525; length generated = 20;
This is unlabeled data 1393!
length=133; hm_slices=20;m=6; b=6;e=7
length origin= 133; length generated = 20;
length=210; hm_slices=20;m=10; b=5;e=5
length origin= 210; length generated = 20;
length=159; hm_slices=20;m=7; b=9;e=10
length origin= 159; length generated = 20;
length=236; hm_slices=20;m=11; b=8;e=8
length origin= 236; length generated = 20;
length=149; hm_slices=20;m=7; b=4;e=5
length origin= 149; length generated = 20;
length=134; hm_slices=20;m=6; b=7;e=7
length origin= 134; length generated = 20;
1400
length=150; hm_slices=20;m=7; b=5;e=5
length origin= 150; length generated = 20;
length=183; hm_slices=20;m=9; b=1;e=2
length origin= 183; length generated = 20;
length=156; hm_slices=20;m=7; b=8;e=8
length origin= 156; length generated = 20;
length=121; hm_slices=20;m=6; b=0;e=1
length origin= 121; length generated = 20;
This is unlabeled data 1404!
length=163; hm_slices=20;m=8; b=1;e=2
length origin= 163; length generated = 20;
length=224; hm_slices=20;m=11; b=2;e=2
length origin= 224; length generated = 20;
length=151; hm_slices=20;m=7; b=5;e=6
length origin= 151; length generated = 20;
length=155; hm_slices=20;m=7; b=7;e=8
length origin= 155; length generated = 20;
length=197; hm_slices=20;m=9; b=8;e=9
length origin= 197; length generated = 20;
length=130; hm_slices=20;m=6; b=5;e=5
length origin= 130; length generated = 20;
length=153; hm_slices=20;m=7; b=6;e=7
length origin= 153; length generated = 20;
length=114; hm_slices=20;m=5; b=7;e=7
length origin= 114; length generated = 20;
length=125; hm_slices=20;m=6; b=2;e=3
length origin= 125; length generated = 20;
length=166; hm_slices=20;m=8; b=3;e=3
length origin= 166; length generated = 20;
length=173; hm_slices=20;m=8; b=6;e=7
length origin= 173; length generated = 20;
length=161; hm_slices=20;m=8; b=0;e=1
length origin= 161; length generated = 20;
length=180; hm_slices=20;m=9; b=0;e=0
length origin= 180; length generated = 20;
length=192; hm_slices=20;m=9; b=6;e=6
length origin= 192; length generated = 20;
This is unlabeled data 1419!
length=123; hm_slices=20;m=6; b=1;e=2
length origin= 123; length generated = 20;
length=131; hm_slices=20;m=6; b=5;e=6
length origin= 131; length generated = 20;
length=155; hm_slices=20;m=7; b=7;e=8
length origin= 155; length generated = 20;
length=153; hm_slices=20;m=7; b=6;e=7
length origin= 153; length generated = 20;
This is unlabeled data 1424!
This is unlabeled data 1425!
length=219; hm_slices=20;m=10; b=9;e=10
length origin= 219; length generated = 20;
length=119; hm_slices=20;m=5; b=9;e=10
length origin= 119; length generated = 20;
length=195; hm_slices=20;m=9; b=7;e=8
length origin= 195; length generated = 20;
length=152; hm_slices=20;m=7; b=6;e=6
length origin= 152; length generated = 20;
This is unlabeled data 1430!
length=435; hm_slices=20;m=21; b=7;e=8
length origin= 435; length generated = 20;
length=296; hm_slices=20;m=14; b=8;e=8
length origin= 296; length generated = 20;
This is unlabeled data 1433!
length=135; hm_slices=20;m=6; b=7;e=8
length origin= 135; length generated = 20;
This is unlabeled data 1435!

length=176; hm_slices=20;m=8; b=8;e=8
length origin= 176; length generated = 20;
length=156; hm_slices=20;m=7; b=8;e=8
length origin= 156; length generated = 20;
length=182; hm_slices=20;m=9; b=1;e=1
length origin= 182; length generated = 20;
length=134; hm_slices=20;m=6; b=7;e=7
length origin= 134; length generated = 20;
length=140; hm_slices=20;m=7; b=0;e=0
length origin= 140; length generated = 20;
length=169; hm_slices=20;m=8; b=4;e=5
length origin= 169; length generated = 20;
This is unlabeled data 1442!
This is unlabeled data 1443!
length=127; hm_slices=20;m=6; b=3;e=4
length origin= 127; length generated = 20;
length=126; hm_slices=20;m=6; b=3;e=3
length origin= 126; length generated = 20;
length=133; hm_slices=20;m=6; b=6;e=7
length origin= 133; length generated = 20;
length=160; hm_slices=20;m=8; b=0;e=0
length origin= 160; length generated = 20;
length=287; hm_slices=20;m=14; b=3;e=4
length origin= 287; length generated = 20;
length=153; hm_slices=20;m=7; b=6;e=7
length origin= 153; length generated = 20;
length=231; hm_slices=20;m=11; b=5;e=6
length origin= 231; length generated = 20;
This is unlabeled data 1451!
length=168; hm_slices=20;m=8; b=4;e=4
length origin= 168; length generated = 20;
length=130; hm_slices=20;m=6; b=5;e=5
length origin= 130; length generated = 20;
length=151; hm_slices=20;m=7; b=5;e=6
length origin= 151; length generated = 20;
length=278; hm_slices=20;m=13; b=9;e=9
length origin= 278; length generated = 20;
length=166; hm_slices=20;m=8; b=3;e=3
length origin= 166; length generated = 20;
length=108; hm_slices=20;m=5; b=4;e=4
length origin= 108; length generated = 20;
length=130; hm_slices=20;m=6; b=5;e=5
length origin= 130; length generated = 20;
length=141; hm_slices=20;m=7; b=0;e=1
length origin= 141; length generated = 20;
length=210; hm_slices=20;m=10; b=5;e=5
length origin= 210; length generated = 20;
length=174; hm_slices=20;m=8; b=7;e=7
length origin= 174; length generated = 20;
length=125; hm_slices=20;m=6; b=2;e=3
length origin= 125; length generated = 20;
This is unlabeled data 1463!
length=124; hm_slices=20;m=6; b=2;e=2
length origin= 124; length generated = 20;
length=232; hm_slices=20;m=11; b=6;e=6
length origin= 232; length generated = 20;
This is unlabeled data 1466!
length=177; hm_slices=20;m=8; b=8;e=9
length origin= 177; length generated = 20;
This is unlabeled data 1468!
length=157; hm_slices=20;m=7; b=8;e=9
length origin= 157; length generated = 20;
length=119; hm_slices=20;m=5; b=9;e=10
length origin= 119; length generated = 20;
length=157; hm_slices=20;m=7; b=8;e=9
length origin= 157; length generated = 20;
length=160; hm_slices=20;m=8; b=0;e=0
length origin= 160; length generated = 20;
length=208; hm_slices=20;m=10; b=4;e=4
length origin= 208; length generated = 20;
length=168; hm_slices=20;m=8; b=4;e=4
length origin= 168; length generated = 20;
length=179; hm_slices=20;m=8; b=9;e=10
length origin= 179; length generated = 20;
length=131; hm_slices=20;m=6; b=5;e=6
length origin= 131; length generated = 20;
length=165; hm_slices=20;m=8; b=2;e=3
length origin= 165; length generated = 20;
length=156; hm_slices=20;m=7; b=8;e=8
length origin= 156; length generated = 20;
This is unlabeled data 1479!
length=240; hm_slices=20;m=12; b=0;e=0
length origin= 240; length generated = 20;
length=281; hm_slices=20;m=14; b=0;e=1
length origin= 281; length generated = 20;
This is unlabeled data 1482!
length=127; hm_slices=20;m=6; b=3;e=4
length origin= 127; length generated = 20;
length=201; hm_slices=20;m=10; b=0;e=1
length origin= 201; length generated = 20;
length=213; hm_slices=20;m=10; b=6;e=7
length origin= 213; length generated = 20;
length=193; hm_slices=20;m=9; b=6;e=7
length origin= 193; length generated = 20;
length=162; hm_slices=20;m=8; b=1;e=1
length origin= 162; length generated = 20;
length=147; hm_slices=20;m=7; b=3;e=4
length origin= 147; length generated = 20;
length=128; hm_slices=20;m=6; b=4;e=4
length origin= 128; length generated = 20;
length=151; hm_slices=20;m=7; b=5;e=6
length origin= 151; length generated = 20;
length=118; hm_slices=20;m=5; b=9;e=9
length origin= 118; length generated = 20;
length=154; hm_slices=20;m=7; b=7;e=7
length origin= 154; length generated = 20;
length=348; hm_slices=20;m=17; b=4;e=4
length origin= 348; length generated = 20;
length=166; hm_slices=20;m=8; b=3;e=3
length origin= 166; length generated = 20;
length=127; hm_slices=20;m=6; b=3;e=4
length origin= 127; length generated = 20;
length=134; hm_slices=20;m=6; b=7;e=7
length origin= 134; length generated = 20;
length=155; hm_slices=20;m=7; b=7;e=8
length origin= 155; length generated = 20;
length=434; hm_slices=20;m=21; b=7;e=7
length origin= 434; length generated = 20;
length=146; hm_slices=20;m=7; b=3;e=3
length origin= 146; length generated = 20;
1500
length=131; hm_slices=20;m=6; b=5;e=6
length origin= 131; length generated = 20;
length=135; hm_slices=20;m=6; b=7;e=8
length origin= 135; length generated = 20;
length=172; hm_slices=20;m=8; b=6;e=6
length origin= 172; length generated = 20;
This is unlabeled data 1503!
length=175; hm_slices=20;m=8; b=7;e=8
length origin= 175; length generated = 20;
length=165; hm_slices=20;m=8; b=2;e=3
length origin= 165; length generated = 20;
This is unlabeled data 1506!
length=166; hm_slices=20;m=8; b=3;e=3
length origin= 166; length generated = 20;
length=155; hm_slices=20;m=7; b=7;e=8
length origin= 155; length generated = 20;
length=239; hm_slices=20;m=11; b=9;e=10
length origin= 239; length generated = 20;
length=234; hm_slices=20;m=11; b=7;e=7
length origin= 234; length generated = 20;
length=230; hm_slices=20;m=11; b=5;e=5
length origin= 230; length generated = 20;
length=472; hm_slices=20;m=23; b=6;e=6
length origin= 472; length generated = 20;
length=155; hm_slices=20;m=7; b=7;e=8
length origin= 155; length generated = 20;
This is unlabeled data 1514!
length=145; hm_slices=20;m=7; b=2;e=3
length origin= 145; length generated = 20;
length=174; hm_slices=20;m=8; b=7;e=7
length origin= 174; length generated = 20;
length=229; hm_slices=20;m=11; b=4;e=5
length origin= 229; length generated = 20;
length=253; hm_slices=20;m=12; b=6;e=7
length origin= 253; length generated = 20;
length=135; hm_slices=20;m=6; b=7;e=8
length origin= 135; length generated = 20;
length=115; hm_slices=20;m=5; b=7;e=8
length origin= 115; length generated = 20;
length=99; hm_slices=20;m=4; b=9;e=10
length origin= 99; length generated = 23;
((23, 50, 50), array([1, 0]))
length=167; hm_slices=20;m=8; b=3;e=4
length origin= 167; length generated = 20;
length=156; hm_slices=20;m=7; b=8;e=8
length origin= 156; length generated = 20;
length=180; hm_slices=20;m=9; b=0;e=0
length origin= 180; length generated = 20;
length=116; hm_slices=20;m=5; b=8;e=8
length origin= 116; length generated = 20;
length=159; hm_slices=20;m=7; b=9;e=10
length origin= 159; length generated = 20;
length=109; hm_slices=20;m=5; b=4;e=5
length origin= 109; length generated = 20;
length=218; hm_slices=20;m=10; b=9;e=9
length origin= 218; length generated = 20;
length=140; hm_slices=20;m=7; b=0;e=0
length origin= 140; length generated = 20;
length=114; hm_slices=20;m=5; b=7;e=7
length origin= 114; length generated = 20;
length=175; hm_slices=20;m=8; b=7;e=8
length origin= 175; length generated = 20;
length=245; hm_slices=20;m=12; b=2;e=3
length origin= 245; length generated = 20;
This is unlabeled data 1533!
length=265; hm_slices=20;m=13; b=2;e=3
length origin= 265; length generated = 20;
length=294; hm_slices=20;m=14; b=7;e=7
length origin= 294; length generated = 20;
length=163; hm_slices=20;m=8; b=1;e=2
length origin= 163; length generated = 20;
length=152; hm_slices=20;m=7; b=6;e=6
length origin= 152; length generated = 20;
length=189; hm_slices=20;m=9; b=4;e=5
length origin= 189; length generated = 20;
length=219; hm_slices=20;m=10; b=9;e=10
length origin= 219; length generated = 20;
length=167; hm_slices=20;m=8; b=3;e=4
length origin= 167; length generated = 20;
length=170; hm_slices=20;m=8; b=5;e=5
length origin= 170; length generated = 20;
length=175; hm_slices=20;m=8; b=7;e=8
length origin= 175; length generated = 20;
length=191; hm_slices=20;m=9; b=5;e=6
length origin= 191; length generated = 20;
length=338; hm_slices=20;m=16; b=9;e=9
length origin= 338; length generated = 20;

length=379; hm_slices=20;m=18; b=9;e=10
length origin= 379; length generated = 20;
length=115; hm_slices=20;m=5; b=7;e=8
length origin= 115; length generated = 20;
This is unlabeled data 1547!
length=216; hm_slices=20;m=10; b=8;e=8
length origin= 216; length generated = 20;
length=140; hm_slices=20;m=7; b=0;e=0
length origin= 140; length generated = 20;
length=145; hm_slices=20;m=7; b=2;e=3
length origin= 145; length generated = 20;
length=252; hm_slices=20;m=12; b=6;e=6
length origin= 252; length generated = 20;
length=175; hm_slices=20;m=8; b=7;e=8
length origin= 175; length generated = 20;
length=250; hm_slices=20;m=12; b=5;e=5
length origin= 250; length generated = 20;
length=145; hm_slices=20;m=7; b=2;e=3
length origin= 145; length generated = 20;
length=133; hm_slices=20;m=6; b=6;e=7
length origin= 133; length generated = 20;
length=153; hm_slices=20;m=7; b=6;e=7
length origin= 153; length generated = 20;
length=271; hm_slices=20;m=13; b=5;e=6
length origin= 271; length generated = 20;
length=191; hm_slices=20;m=9; b=5;e=6
length origin= 191; length generated = 20;
This is unlabeled data 1559!
This is unlabeled data 1560!
length=184; hm_slices=20;m=9; b=2;e=2
length origin= 184; length generated = 20;
length=153; hm_slices=20;m=7; b=6;e=7
length origin= 153; length generated = 20;
length=192; hm_slices=20;m=9; b=6;e=6
length origin= 192; length generated = 20;
length=161; hm_slices=20;m=8; b=0;e=1
length origin= 161; length generated = 20;
length=128; hm_slices=20;m=6; b=4;e=4
length origin= 128; length generated = 20;
length=161; hm_slices=20;m=8; b=0;e=1
length origin= 161; length generated = 20;
length=261; hm_slices=20;m=13; b=0;e=1
length origin= 261; length generated = 20;
length=106; hm_slices=20;m=5; b=3;e=3
length origin= 106; length generated = 20;
length=132; hm_slices=20;m=6; b=6;e=6
length origin= 132; length generated = 20;
length=135; hm_slices=20;m=6; b=7;e=8
length origin= 135; length generated = 20;
length=161; hm_slices=20;m=8; b=0;e=1
length origin= 161; length generated = 20;
This is unlabeled data 1572!
length=172; hm_slices=20;m=8; b=6;e=6
length origin= 172; length generated = 20;
length=135; hm_slices=20;m=6; b=7;e=8
length origin= 135; length generated = 20;
length=186; hm_slices=20;m=9; b=3;e=3
length origin= 186; length generated = 20;
length=135; hm_slices=20;m=6; b=7;e=8
length origin= 135; length generated = 20;
length=148; hm_slices=20;m=7; b=4;e=4
length origin= 148; length generated = 20;
length=208; hm_slices=20;m=10; b=4;e=4
length origin= 208; length generated = 20;
length=181; hm_slices=20;m=9; b=0;e=1
length origin= 181; length generated = 20;
length=170; hm_slices=20;m=8; b=5;e=5
length origin= 170; length generated = 20;
length=123; hm_slices=20;m=6; b=1;e=2
length origin= 123; length generated = 20;
length=116; hm_slices=20;m=5; b=8;e=8
length origin= 116; length generated = 20;
length=209; hm_slices=20;m=10; b=4;e=5
length origin= 209; length generated = 20;
length=166; hm_slices=20;m=8; b=3;e=3
length origin= 166; length generated = 20;
length=107; hm_slices=20;m=5; b=3;e=4
length origin= 107; length generated = 20;
length=112; hm_slices=20;m=5; b=6;e=6
length origin= 112; length generated = 20;
length=156; hm_slices=20;m=7; b=8;e=8
length origin= 156; length generated = 20;
length=167; hm_slices=20;m=8; b=3;e=4
length origin= 167; length generated = 20;
length=186; hm_slices=20;m=9; b=3;e=3
length origin= 186; length generated = 20;
length=170; hm_slices=20;m=8; b=5;e=5
length origin= 170; length generated = 20;
length=160; hm_slices=20;m=8; b=0;e=0
length origin= 160; length generated = 20;
length=166; hm_slices=20;m=8; b=3;e=3
length origin= 166; length generated = 20;
This is unlabeled data 1593!
length=148; hm_slices=20;m=7; b=4;e=4
length origin= 148; length generated = 20;
